## Improved GAN: feature matching

GAN for semi-supervised learning: feature-matching technique. NN constructed according to "later" implementation.

*Later implementation (2018):*

Original code (tf): https://github.com/bruno-31/GAN-manifold-regularization

Paper: https://openreview.net/forum?id=Hy5QRt1wz

*Original implementation (2016):*

Original code (theano): https://github.com/openai/improved-gan

Paper: https://arxiv.org/abs/1606.03498

**Not implemented or different in comparison with original NN: **
* No data-based initialization (from 100 training examples for stats' calculation)
* No learning rate decay (from 1200 to 1400 epochs)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import torch.nn.functional as F
import matplotlib.image as mpimg
import numpy as np

from tqdm import tqdm
from torch.nn.utils import weight_norm
from torch.autograd import Variable
from utils import get_data_loaders, plot_grid
from utils import log_sum_exp

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
lr = 0.0003
train_size = 400
batch_size = 25
z_dim = 100
ema_coeff = 0.0001
n = 32    # how to resize image
num_channels = 3   # img channels
use_cuda = torch.cuda.is_available()
print('use_cuda: {}'.format(use_cuda))

In [ ]:
transform = transforms.Compose([transforms.Resize((n, n)), transforms.ToTensor(), 
                              transforms.Lambda(lambda x: x * 2. - 1.)])

In [ ]:
train_loader1, test_loader = get_data_loaders('cifar10', transform=transform, batch_size=batch_size, 
                                             use_cuda=use_cuda)
train_loader2, test_loader = get_data_loaders('cifar10', transform=transform, batch_size=batch_size, 
                                             use_cuda=use_cuda)

In [ ]:
train_loader_ssl, _ = get_data_loaders('cifar10', transform=transform, batch_size=batch_size, 
                                             use_cuda=use_cuda, train_size=400)

In [ ]:
def weights_init(model):
    classname = model.__class__.__name__
    if classname.find('Conv2d') != -1:
        model.weight.data.normal_(0.0, 0.05)
        model.bias.data.fill_(0)
        if classname.find('weight_g') != -1:
            model.weight_g.data.fill_(1)
    elif classname.find('Linear') != -1:
        model.weight.data.normal_(0.0, 0.05)
        model.bias.data.fill_(0)
        if classname.find('weight_g') != -1:
            model.weight_g.data.fill_(1)
    elif classname.find('ConvTranspose2d') != -1:
        model.weight.data.normal_(0.0, 0.05)
        model.bias.data.fill_(0)
        if classname.find('weight_g') != -1:
            model.weight_g.data.fill_(1)

In [ ]:
def test(net, testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs, _ = net(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = correct / total
    net.train()
    return acc

In [ ]:
def update_avg_nn(model_avg, model, coeff):
    for (na, pa), (n, p) in zip(model_avg.named_parameters(), model.named_parameters()):
        pa.data = pa.data * (1 - coeff) + coeff * p.data

In [ ]:
class Dc(nn.Module):
    def __init__(self, num_channels=3, img_size=(32, 32), num_classes=10):
        super(Dc, self).__init__()
        self.conv = nn.Sequential(
            nn.Dropout(0.2),
            # block 1
            weight_norm(nn.Conv2d(in_channels=num_channels, out_channels=96, kernel_size=3, padding=1)),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=96, out_channels=96, kernel_size=3, padding=1)),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=96, out_channels=96, kernel_size=3, stride=2, padding=1)),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.5),
            # block 2
            weight_norm(nn.Conv2d(in_channels=96, out_channels=192, kernel_size=3, padding=1)),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, padding=1)),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, stride=2, padding=1)),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.5),
            # block 3
            weight_norm(nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3)),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=192, out_channels=192, kernel_size=1)),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=192, out_channels=192, kernel_size=1)),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(kernel_size=6, stride=1, padding=0), 
        )
        self.logits = weight_norm(nn.Linear(192, num_classes))
        
    def forward(self, x):
        x = self.conv(x)
        features = x.view(-1, x.size(1))
        logits = self.logits(features)
        return logits, features

In [ ]:
class Gc(nn.Module):
    def __init__(self, dim_z, dim_features=512, num_channels=3):
        super(Gc, self).__init__()
        self.dim_features = dim_features
        self.num_channels = num_channels
        self.l1 = nn.Sequential(
            nn.Linear(dim_z, self.dim_features * 4 * 4),
            nn.BatchNorm1d(self.dim_features * 4 * 4),
            nn.ReLU(),
        )
        self.l2 = nn.Sequential(
            nn.ConvTranspose2d(self.dim_features, self.dim_features // 2, kernel_size=5, stride=2, 
                               output_padding=1, padding=2),
            nn.BatchNorm2d(self.dim_features // 2),
            nn.ReLU(),
            nn.ConvTranspose2d(self.dim_features // 2, self.dim_features // 4, kernel_size=5, stride=2, 
                                output_padding=1, padding=2),
            nn.BatchNorm2d(self.dim_features // 4),
            nn.ReLU(),
            weight_norm(nn.ConvTranspose2d(self.dim_features // 4, self.num_channels, kernel_size=5, stride=2, 
                                output_padding=1, padding=2)),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = self.l1(x).view(x.size(0), self.dim_features, 4, 4)
        img = self.l2(x)
        return img   

In [ ]:
D = Dc()
D_avg = Dc()
G = Gc(z_dim)
D.train(), G.train(), D_avg.train()
print()

In [ ]:
D.apply(weights_init)
G.apply(weights_init)
D_avg.load_state_dict(D.state_dict())
print()

In [ ]:
softplus = nn.Softplus()
criterion = nn.CrossEntropyLoss()
if use_cuda:
    D.cuda()
    G.cuda()
    D_avg.cuda()

In [ ]:
D_opt = optim.Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))
G_opt = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:
# r = real, f = fake, ul = unlabelled, s = sample
final_acc = []
print()
D.zero_grad()
G.zero_grad()
for ne in range(1000):
    c = 0
    it = iter(train_loader_ssl)
    eloss, gloss = [], []
    for (x_ul_1, _), (x_ul_2, _) in zip(train_loader1, train_loader2):
        # setting up input data
        c += 1
        if c == len(train_loader_ssl):  # restarting train_loader_ssl
            c = 0
            it = iter(train_loader_ssl)
        z = torch.randn(batch_size, z_dim)
        x_l, y_l = next(it)
        if use_cuda:
            x_l = x_l.cuda()
            y_l = y_l.cuda()
            x_ul_1 = x_ul_1.cuda()
            x_ul_2 = x_ul_2.cuda()
            z = z.cuda()
        # train discriminator
        D_r_l, _ = D(x_l)
        D_r_ul, _ = D(x_ul_1)
        G_s = G(z)
        D_f, _ = D(G_s.detach())
        loss_l = criterion(D_r_l, y_l)
        ul = log_sum_exp(D_r_ul)
        lf = log_sum_exp(D_f)
        loss_ul = -ul.mean() + softplus(ul).mean() + softplus(lf).mean()
        D_loss = loss_l + 0.5 * loss_ul 
        D_loss.backward()
        D_opt.step()
        D.zero_grad()
        G.zero_grad()
        update_avg_nn(D_avg, D, ema_coeff)         
        # train generator
        _, layer_r = D(x_ul_2)
        _, layer_f = D(G_s)
        G_loss = torch.abs(layer_r.detach().mean(dim=0) - layer_f.mean(dim=0)).mean()
        G_loss.backward()
        G_opt.step()
        D.zero_grad()
        G.zero_grad()
        # stats
        eloss.append(D_loss.data.cpu().numpy())
        gloss.append(G_loss.data.cpu().numpy())
    acc = test(D, test_loader)
    acc_avg = test(D_avg, test_loader)
    final_acc.append(acc_avg)
    print('Epoch: {}; D_loss: {:.2f}; G_loss: {:.2f}; Accuracy: {:.2f}%; Accuracy Avg: {:.2f}%;'\
          .format(ne, np.array(eloss).mean(), np.array(gloss).mean(), acc*100., acc_avg*100.))

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]
G.training = True
z = torch.randn(64, z_dim).cuda() 
vutils.save_image(G(z).data, 'temp.png')
img = mpimg.imread('temp.png')
plt.imshow(img)
G.training = True

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]
l = next(iter(train_loader))
vutils.save_image(l[0], 'temp2.png')
img = mpimg.imread('temp2.png')
plt.imshow(img)
G.training = True

In [ ]:
for n, p in list(D_avg.named_parameters()):
    print(n, p.size())

In [ ]:
# w/o data-driven init
# Epoch: 0; D_loss: 2.53; G_loss: 0.91; Accuracy: 30.49%; Accuracy Avg: 10.00%;
# Epoch: 1; D_loss: 2.24; G_loss: 0.44; Accuracy: 38.48%; Accuracy Avg: 10.00%;
# Epoch: 2; D_loss: 1.93; G_loss: 0.41; Accuracy: 37.69%; Accuracy Avg: 10.34%;
# Epoch: 3; D_loss: 1.73; G_loss: 0.41; Accuracy: 45.26%; Accuracy Avg: 10.76%;
# Epoch: 4; D_loss: 1.55; G_loss: 0.42; Accuracy: 50.75%; Accuracy Avg: 14.40%;
# Epoch: 5; D_loss: 1.39; G_loss: 0.45; Accuracy: 49.58%; Accuracy Avg: 23.26%;
# Epoch: 6; D_loss: 1.23; G_loss: 0.49; Accuracy: 49.94%; Accuracy Avg: 32.78%;
# Epoch: 7; D_loss: 1.09; G_loss: 0.56; Accuracy: 53.14%; Accuracy Avg: 40.93%;
# Epoch: 8; D_loss: 0.98; G_loss: 0.61; Accuracy: 46.74%; Accuracy Avg: 47.18%;
# Epoch: 9; D_loss: 0.86; G_loss: 0.66; Accuracy: 53.91%; Accuracy Avg: 51.32%;
# Epoch: 10; D_loss: 0.78; G_loss: 0.68; Accuracy: 55.94%; Accuracy Avg: 53.50%;
# Epoch: 11; D_loss: 0.71; G_loss: 0.72; Accuracy: 52.77%; Accuracy Avg: 55.01%;
# Epoch: 12; D_loss: 0.66; G_loss: 0.73; Accuracy: 56.80%; Accuracy Avg: 56.00%;
# Epoch: 13; D_loss: 0.62; G_loss: 0.73; Accuracy: 52.67%; Accuracy Avg: 56.80%;
# Epoch: 14; D_loss: 0.58; G_loss: 0.72; Accuracy: 54.37%; Accuracy Avg: 57.46%;
# Epoch: 15; D_loss: 0.55; G_loss: 0.73; Accuracy: 54.15%; Accuracy Avg: 57.92%;
# Epoch: 16; D_loss: 0.53; G_loss: 0.72; Accuracy: 57.40%; Accuracy Avg: 58.33%;
# Epoch: 17; D_loss: 0.51; G_loss: 0.71; Accuracy: 55.70%; Accuracy Avg: 58.76%;
# Epoch: 18; D_loss: 0.49; G_loss: 0.71; Accuracy: 59.39%; Accuracy Avg: 59.04%;
# Epoch: 19; D_loss: 0.48; G_loss: 0.69; Accuracy: 56.73%; Accuracy Avg: 59.41%;
# Epoch: 20; D_loss: 0.46; G_loss: 0.69; Accuracy: 56.87%; Accuracy Avg: 59.58%;
# Epoch: 21; D_loss: 0.46; G_loss: 0.68; Accuracy: 56.70%; Accuracy Avg: 59.51%;
# Epoch: 22; D_loss: 0.44; G_loss: 0.67; Accuracy: 56.59%; Accuracy Avg: 59.47%;
# Epoch: 23; D_loss: 0.44; G_loss: 0.65; Accuracy: 60.56%; Accuracy Avg: 59.75%;
# Epoch: 24; D_loss: 0.43; G_loss: 0.64; Accuracy: 57.51%; Accuracy Avg: 59.87%;
# Epoch: 25; D_loss: 0.42; G_loss: 0.63; Accuracy: 59.96%; Accuracy Avg: 60.06%;
# Epoch: 26; D_loss: 0.42; G_loss: 0.62; Accuracy: 59.97%; Accuracy Avg: 60.13%;
# Epoch: 27; D_loss: 0.41; G_loss: 0.60; Accuracy: 60.59%; Accuracy Avg: 60.32%;
# Epoch: 28; D_loss: 0.42; G_loss: 0.59; Accuracy: 61.12%; Accuracy Avg: 60.50%;
# Epoch: 29; D_loss: 0.40; G_loss: 0.58; Accuracy: 58.13%; Accuracy Avg: 60.78%;
# Epoch: 30; D_loss: 0.40; G_loss: 0.57; Accuracy: 59.76%; Accuracy Avg: 60.89%;
# Epoch: 31; D_loss: 0.39; G_loss: 0.56; Accuracy: 61.14%; Accuracy Avg: 61.07%;
# Epoch: 32; D_loss: 0.39; G_loss: 0.55; Accuracy: 60.73%; Accuracy Avg: 61.25%;
# Epoch: 33; D_loss: 0.39; G_loss: 0.55; Accuracy: 61.07%; Accuracy Avg: 61.35%;
# Epoch: 34; D_loss: 0.38; G_loss: 0.54; Accuracy: 58.63%; Accuracy Avg: 61.45%;
# Epoch: 35; D_loss: 0.37; G_loss: 0.54; Accuracy: 59.93%; Accuracy Avg: 61.69%;
# Epoch: 36; D_loss: 0.37; G_loss: 0.53; Accuracy: 57.44%; Accuracy Avg: 61.80%;
# Epoch: 37; D_loss: 0.37; G_loss: 0.52; Accuracy: 63.14%; Accuracy Avg: 61.81%;
# Epoch: 38; D_loss: 0.36; G_loss: 0.51; Accuracy: 62.11%; Accuracy Avg: 61.70%;
# Epoch: 39; D_loss: 0.36; G_loss: 0.51; Accuracy: 60.07%; Accuracy Avg: 61.88%;
# Epoch: 40; D_loss: 0.36; G_loss: 0.49; Accuracy: 58.33%; Accuracy Avg: 62.01%;
# Epoch: 41; D_loss: 0.36; G_loss: 0.50; Accuracy: 61.52%; Accuracy Avg: 62.16%;
# Epoch: 42; D_loss: 0.36; G_loss: 0.49; Accuracy: 59.07%; Accuracy Avg: 62.20%;
# Epoch: 43; D_loss: 0.36; G_loss: 0.48; Accuracy: 63.49%; Accuracy Avg: 62.25%;
# Epoch: 44; D_loss: 0.35; G_loss: 0.47; Accuracy: 61.94%; Accuracy Avg: 62.30%;
# Epoch: 45; D_loss: 0.35; G_loss: 0.47; Accuracy: 62.47%; Accuracy Avg: 62.55%;
# Epoch: 46; D_loss: 0.35; G_loss: 0.46; Accuracy: 61.42%; Accuracy Avg: 62.65%;
# Epoch: 47; D_loss: 0.34; G_loss: 0.46; Accuracy: 62.43%; Accuracy Avg: 62.74%;
# Epoch: 48; D_loss: 0.34; G_loss: 0.45; Accuracy: 58.82%; Accuracy Avg: 62.84%;
# Epoch: 49; D_loss: 0.33; G_loss: 0.46; Accuracy: 61.76%; Accuracy Avg: 62.87%;
# Epoch: 50; D_loss: 0.34; G_loss: 0.45; Accuracy: 59.97%; Accuracy Avg: 62.85%;
# Epoch: 51; D_loss: 0.34; G_loss: 0.44; Accuracy: 63.44%; Accuracy Avg: 63.04%;
# Epoch: 52; D_loss: 0.33; G_loss: 0.44; Accuracy: 63.17%; Accuracy Avg: 63.25%;
# Epoch: 53; D_loss: 0.32; G_loss: 0.44; Accuracy: 63.90%; Accuracy Avg: 63.38%;
# Epoch: 54; D_loss: 0.33; G_loss: 0.43; Accuracy: 63.08%; Accuracy Avg: 63.61%;
# Epoch: 55; D_loss: 0.32; G_loss: 0.43; Accuracy: 62.22%; Accuracy Avg: 63.76%;
# Epoch: 56; D_loss: 0.33; G_loss: 0.42; Accuracy: 61.36%; Accuracy Avg: 63.70%;
# Epoch: 57; D_loss: 0.32; G_loss: 0.42; Accuracy: 64.44%; Accuracy Avg: 63.72%;
# Epoch: 58; D_loss: 0.32; G_loss: 0.42; Accuracy: 62.29%; Accuracy Avg: 63.74%;
# Epoch: 59; D_loss: 0.31; G_loss: 0.42; Accuracy: 63.69%; Accuracy Avg: 63.82%;
# Epoch: 60; D_loss: 0.31; G_loss: 0.41; Accuracy: 63.87%; Accuracy Avg: 63.81%;
# Epoch: 61; D_loss: 0.31; G_loss: 0.41; Accuracy: 64.32%; Accuracy Avg: 63.85%;
# Epoch: 62; D_loss: 0.31; G_loss: 0.40; Accuracy: 63.85%; Accuracy Avg: 64.18%;
# Epoch: 63; D_loss: 0.31; G_loss: 0.40; Accuracy: 64.41%; Accuracy Avg: 64.32%;
# Epoch: 64; D_loss: 0.31; G_loss: 0.40; Accuracy: 64.69%; Accuracy Avg: 64.57%;
# Epoch: 65; D_loss: 0.31; G_loss: 0.39; Accuracy: 63.06%; Accuracy Avg: 64.77%;
# Epoch: 66; D_loss: 0.30; G_loss: 0.40; Accuracy: 63.58%; Accuracy Avg: 64.82%;
# Epoch: 67; D_loss: 0.31; G_loss: 0.39; Accuracy: 64.78%; Accuracy Avg: 64.87%;
# Epoch: 68; D_loss: 0.30; G_loss: 0.39; Accuracy: 66.82%; Accuracy Avg: 64.94%;
# Epoch: 69; D_loss: 0.30; G_loss: 0.38; Accuracy: 64.67%; Accuracy Avg: 65.08%;
# Epoch: 70; D_loss: 0.30; G_loss: 0.39; Accuracy: 65.62%; Accuracy Avg: 65.36%;
# Epoch: 71; D_loss: 0.30; G_loss: 0.38; Accuracy: 64.21%; Accuracy Avg: 65.55%;
# Epoch: 72; D_loss: 0.30; G_loss: 0.38; Accuracy: 64.51%; Accuracy Avg: 65.66%;
# Epoch: 73; D_loss: 0.30; G_loss: 0.37; Accuracy: 66.45%; Accuracy Avg: 65.80%;
# Epoch: 74; D_loss: 0.29; G_loss: 0.38; Accuracy: 67.12%; Accuracy Avg: 65.88%;
# Epoch: 75; D_loss: 0.29; G_loss: 0.37; Accuracy: 67.01%; Accuracy Avg: 65.99%;
# Epoch: 76; D_loss: 0.29; G_loss: 0.37; Accuracy: 68.33%; Accuracy Avg: 66.41%;
# Epoch: 77; D_loss: 0.29; G_loss: 0.36; Accuracy: 64.40%; Accuracy Avg: 66.65%;
# Epoch: 78; D_loss: 0.29; G_loss: 0.37; Accuracy: 64.85%; Accuracy Avg: 66.80%;
# Epoch: 79; D_loss: 0.29; G_loss: 0.36; Accuracy: 65.32%; Accuracy Avg: 66.84%;
# Epoch: 80; D_loss: 0.29; G_loss: 0.36; Accuracy: 67.73%; Accuracy Avg: 67.05%;
# Epoch: 81; D_loss: 0.29; G_loss: 0.36; Accuracy: 66.33%; Accuracy Avg: 67.05%;
# Epoch: 82; D_loss: 0.28; G_loss: 0.36; Accuracy: 66.94%; Accuracy Avg: 67.26%;
# Epoch: 83; D_loss: 0.28; G_loss: 0.36; Accuracy: 67.33%; Accuracy Avg: 67.38%;
# Epoch: 84; D_loss: 0.28; G_loss: 0.35; Accuracy: 67.17%; Accuracy Avg: 67.62%;
# Epoch: 85; D_loss: 0.28; G_loss: 0.35; Accuracy: 69.56%; Accuracy Avg: 67.82%;
# Epoch: 86; D_loss: 0.28; G_loss: 0.35; Accuracy: 66.61%; Accuracy Avg: 67.90%;
# Epoch: 87; D_loss: 0.27; G_loss: 0.35; Accuracy: 67.88%; Accuracy Avg: 68.06%;
# Epoch: 88; D_loss: 0.28; G_loss: 0.35; Accuracy: 67.78%; Accuracy Avg: 68.19%;
# Epoch: 89; D_loss: 0.28; G_loss: 0.34; Accuracy: 67.87%; Accuracy Avg: 68.28%;
# Epoch: 90; D_loss: 0.27; G_loss: 0.34; Accuracy: 67.57%; Accuracy Avg: 68.58%;
# Epoch: 91; D_loss: 0.28; G_loss: 0.34; Accuracy: 67.42%; Accuracy Avg: 68.67%;
# Epoch: 92; D_loss: 0.28; G_loss: 0.34; Accuracy: 67.41%; Accuracy Avg: 68.88%;
# Epoch: 93; D_loss: 0.27; G_loss: 0.34; Accuracy: 67.15%; Accuracy Avg: 69.01%;
# Epoch: 94; D_loss: 0.27; G_loss: 0.34; Accuracy: 69.94%; Accuracy Avg: 69.26%;
# Epoch: 95; D_loss: 0.27; G_loss: 0.33; Accuracy: 67.71%; Accuracy Avg: 69.30%;
# Epoch: 96; D_loss: 0.27; G_loss: 0.33; Accuracy: 68.42%; Accuracy Avg: 69.47%;
# Epoch: 97; D_loss: 0.27; G_loss: 0.33; Accuracy: 69.71%; Accuracy Avg: 69.50%;
# Epoch: 98; D_loss: 0.26; G_loss: 0.33; Accuracy: 68.14%; Accuracy Avg: 69.66%;
# Epoch: 99; D_loss: 0.26; G_loss: 0.33; Accuracy: 68.90%; Accuracy Avg: 69.69%;
# Epoch: 100; D_loss: 0.26; G_loss: 0.33; Accuracy: 67.98%; Accuracy Avg: 69.68%;
# Epoch: 101; D_loss: 0.27; G_loss: 0.32; Accuracy: 67.70%; Accuracy Avg: 69.77%;
# Epoch: 102; D_loss: 0.26; G_loss: 0.32; Accuracy: 68.30%; Accuracy Avg: 69.82%;
# Epoch: 103; D_loss: 0.27; G_loss: 0.32; Accuracy: 68.02%; Accuracy Avg: 70.08%;
# Epoch: 104; D_loss: 0.26; G_loss: 0.33; Accuracy: 68.95%; Accuracy Avg: 70.03%;
# Epoch: 105; D_loss: 0.26; G_loss: 0.32; Accuracy: 68.95%; Accuracy Avg: 70.11%;
# Epoch: 106; D_loss: 0.26; G_loss: 0.32; Accuracy: 66.57%; Accuracy Avg: 70.27%;
# Epoch: 107; D_loss: 0.26; G_loss: 0.32; Accuracy: 69.30%; Accuracy Avg: 70.48%;
# Epoch: 108; D_loss: 0.26; G_loss: 0.32; Accuracy: 70.50%; Accuracy Avg: 70.47%;
# Epoch: 109; D_loss: 0.26; G_loss: 0.31; Accuracy: 70.68%; Accuracy Avg: 70.66%;
# Epoch: 110; D_loss: 0.26; G_loss: 0.31; Accuracy: 70.70%; Accuracy Avg: 70.70%;
# Epoch: 111; D_loss: 0.26; G_loss: 0.31; Accuracy: 67.60%; Accuracy Avg: 70.67%;
# Epoch: 112; D_loss: 0.26; G_loss: 0.31; Accuracy: 69.19%; Accuracy Avg: 70.76%;
# Epoch: 113; D_loss: 0.26; G_loss: 0.31; Accuracy: 68.74%; Accuracy Avg: 70.83%;
# Epoch: 114; D_loss: 0.25; G_loss: 0.31; Accuracy: 70.82%; Accuracy Avg: 70.86%;
# Epoch: 115; D_loss: 0.25; G_loss: 0.31; Accuracy: 67.95%; Accuracy Avg: 70.87%;
# Epoch: 116; D_loss: 0.25; G_loss: 0.31; Accuracy: 70.90%; Accuracy Avg: 70.94%;
# Epoch: 117; D_loss: 0.25; G_loss: 0.30; Accuracy: 70.11%; Accuracy Avg: 71.04%;
# Epoch: 118; D_loss: 0.25; G_loss: 0.31; Accuracy: 70.31%; Accuracy Avg: 71.20%;
# Epoch: 119; D_loss: 0.25; G_loss: 0.30; Accuracy: 69.61%; Accuracy Avg: 71.31%;
# Epoch: 120; D_loss: 0.25; G_loss: 0.30; Accuracy: 71.58%; Accuracy Avg: 71.51%;
# Epoch: 121; D_loss: 0.25; G_loss: 0.30; Accuracy: 70.85%; Accuracy Avg: 71.48%;
# Epoch: 122; D_loss: 0.25; G_loss: 0.30; Accuracy: 70.57%; Accuracy Avg: 71.67%;
# Epoch: 123; D_loss: 0.25; G_loss: 0.30; Accuracy: 70.31%; Accuracy Avg: 71.84%;
# Epoch: 124; D_loss: 0.25; G_loss: 0.30; Accuracy: 71.72%; Accuracy Avg: 71.93%;
# Epoch: 125; D_loss: 0.25; G_loss: 0.30; Accuracy: 70.72%; Accuracy Avg: 72.05%;
# Epoch: 126; D_loss: 0.25; G_loss: 0.29; Accuracy: 70.03%; Accuracy Avg: 72.14%;
# Epoch: 127; D_loss: 0.25; G_loss: 0.30; Accuracy: 70.65%; Accuracy Avg: 72.29%;
# Epoch: 128; D_loss: 0.24; G_loss: 0.29; Accuracy: 70.09%; Accuracy Avg: 72.37%;
# Epoch: 129; D_loss: 0.25; G_loss: 0.29; Accuracy: 69.59%; Accuracy Avg: 72.29%;
# Epoch: 130; D_loss: 0.24; G_loss: 0.29; Accuracy: 71.71%; Accuracy Avg: 72.39%;
# Epoch: 131; D_loss: 0.24; G_loss: 0.29; Accuracy: 69.26%; Accuracy Avg: 72.46%;
# Epoch: 132; D_loss: 0.24; G_loss: 0.29; Accuracy: 72.00%; Accuracy Avg: 72.53%;
# Epoch: 133; D_loss: 0.24; G_loss: 0.29; Accuracy: 71.51%; Accuracy Avg: 72.59%;
# Epoch: 134; D_loss: 0.24; G_loss: 0.29; Accuracy: 71.89%; Accuracy Avg: 72.65%;
# Epoch: 135; D_loss: 0.24; G_loss: 0.29; Accuracy: 73.07%; Accuracy Avg: 72.68%;
# Epoch: 136; D_loss: 0.24; G_loss: 0.29; Accuracy: 72.56%; Accuracy Avg: 72.73%;
# Epoch: 137; D_loss: 0.24; G_loss: 0.28; Accuracy: 71.39%; Accuracy Avg: 72.84%;
# Epoch: 138; D_loss: 0.24; G_loss: 0.28; Accuracy: 73.98%; Accuracy Avg: 72.94%;
# Epoch: 139; D_loss: 0.24; G_loss: 0.28; Accuracy: 71.47%; Accuracy Avg: 73.11%;
# Epoch: 140; D_loss: 0.23; G_loss: 0.28; Accuracy: 70.50%; Accuracy Avg: 73.07%;
# Epoch: 141; D_loss: 0.24; G_loss: 0.28; Accuracy: 72.00%; Accuracy Avg: 73.00%;
# Epoch: 142; D_loss: 0.23; G_loss: 0.29; Accuracy: 71.46%; Accuracy Avg: 73.04%;
# Epoch: 143; D_loss: 0.24; G_loss: 0.28; Accuracy: 72.56%; Accuracy Avg: 73.07%;
# Epoch: 144; D_loss: 0.23; G_loss: 0.29; Accuracy: 70.87%; Accuracy Avg: 73.17%;
# Epoch: 145; D_loss: 0.24; G_loss: 0.28; Accuracy: 71.44%; Accuracy Avg: 73.19%;
# Epoch: 146; D_loss: 0.23; G_loss: 0.28; Accuracy: 71.33%; Accuracy Avg: 73.35%;
# Epoch: 147; D_loss: 0.23; G_loss: 0.28; Accuracy: 72.60%; Accuracy Avg: 73.33%;
# Epoch: 148; D_loss: 0.23; G_loss: 0.28; Accuracy: 71.85%; Accuracy Avg: 73.47%;
# Epoch: 149; D_loss: 0.24; G_loss: 0.27; Accuracy: 71.39%; Accuracy Avg: 73.62%;
# Epoch: 150; D_loss: 0.23; G_loss: 0.28; Accuracy: 71.71%; Accuracy Avg: 73.73%;
# Epoch: 151; D_loss: 0.23; G_loss: 0.28; Accuracy: 72.45%; Accuracy Avg: 73.74%;
# Epoch: 152; D_loss: 0.23; G_loss: 0.28; Accuracy: 73.07%; Accuracy Avg: 73.86%;
# Epoch: 153; D_loss: 0.23; G_loss: 0.28; Accuracy: 71.51%; Accuracy Avg: 73.95%;
# Epoch: 154; D_loss: 0.23; G_loss: 0.27; Accuracy: 73.39%; Accuracy Avg: 73.99%;
# Epoch: 155; D_loss: 0.23; G_loss: 0.28; Accuracy: 72.07%; Accuracy Avg: 74.01%;
# Epoch: 156; D_loss: 0.23; G_loss: 0.28; Accuracy: 72.90%; Accuracy Avg: 74.23%;
# Epoch: 157; D_loss: 0.23; G_loss: 0.27; Accuracy: 72.01%; Accuracy Avg: 74.20%;
# Epoch: 158; D_loss: 0.22; G_loss: 0.27; Accuracy: 73.77%; Accuracy Avg: 74.22%;
# Epoch: 159; D_loss: 0.22; G_loss: 0.27; Accuracy: 74.41%; Accuracy Avg: 74.08%;
# Epoch: 160; D_loss: 0.22; G_loss: 0.27; Accuracy: 75.09%; Accuracy Avg: 74.14%;
# Epoch: 161; D_loss: 0.22; G_loss: 0.27; Accuracy: 71.72%; Accuracy Avg: 74.22%;
# Epoch: 162; D_loss: 0.22; G_loss: 0.27; Accuracy: 72.15%; Accuracy Avg: 74.24%;
# Epoch: 163; D_loss: 0.22; G_loss: 0.27; Accuracy: 72.44%; Accuracy Avg: 74.17%;
# Epoch: 164; D_loss: 0.22; G_loss: 0.27; Accuracy: 72.38%; Accuracy Avg: 74.14%;
# Epoch: 165; D_loss: 0.21; G_loss: 0.27; Accuracy: 71.60%; Accuracy Avg: 74.27%;
# Epoch: 166; D_loss: 0.22; G_loss: 0.27; Accuracy: 72.49%; Accuracy Avg: 74.28%;
# Epoch: 167; D_loss: 0.22; G_loss: 0.27; Accuracy: 72.56%; Accuracy Avg: 74.25%;
# Epoch: 168; D_loss: 0.22; G_loss: 0.27; Accuracy: 73.89%; Accuracy Avg: 74.38%;
# Epoch: 169; D_loss: 0.22; G_loss: 0.27; Accuracy: 73.15%; Accuracy Avg: 74.43%;
# Epoch: 170; D_loss: 0.21; G_loss: 0.27; Accuracy: 73.21%; Accuracy Avg: 74.49%;
# Epoch: 171; D_loss: 0.21; G_loss: 0.27; Accuracy: 74.02%; Accuracy Avg: 74.66%;
# Epoch: 172; D_loss: 0.21; G_loss: 0.27; Accuracy: 73.78%; Accuracy Avg: 74.64%;
# Epoch: 173; D_loss: 0.21; G_loss: 0.27; Accuracy: 73.55%; Accuracy Avg: 74.80%;
# Epoch: 174; D_loss: 0.21; G_loss: 0.27; Accuracy: 73.42%; Accuracy Avg: 74.83%;
# Epoch: 175; D_loss: 0.21; G_loss: 0.26; Accuracy: 71.73%; Accuracy Avg: 74.95%;
# Epoch: 176; D_loss: 0.21; G_loss: 0.26; Accuracy: 72.07%; Accuracy Avg: 75.08%;
# Epoch: 177; D_loss: 0.21; G_loss: 0.26; Accuracy: 73.77%; Accuracy Avg: 75.25%;
# Epoch: 178; D_loss: 0.21; G_loss: 0.27; Accuracy: 73.49%; Accuracy Avg: 75.22%;
# Epoch: 179; D_loss: 0.22; G_loss: 0.26; Accuracy: 74.22%; Accuracy Avg: 75.29%;
# Epoch: 180; D_loss: 0.22; G_loss: 0.26; Accuracy: 73.95%; Accuracy Avg: 75.34%;
# Epoch: 181; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.13%; Accuracy Avg: 75.37%;
# Epoch: 182; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.86%; Accuracy Avg: 75.30%;
# Epoch: 183; D_loss: 0.21; G_loss: 0.26; Accuracy: 71.87%; Accuracy Avg: 75.34%;
# Epoch: 184; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.95%; Accuracy Avg: 75.35%;
# Epoch: 185; D_loss: 0.21; G_loss: 0.26; Accuracy: 72.99%; Accuracy Avg: 75.42%;
# Epoch: 186; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.77%; Accuracy Avg: 75.36%;
# Epoch: 187; D_loss: 0.21; G_loss: 0.26; Accuracy: 73.35%; Accuracy Avg: 75.46%;
# Epoch: 188; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.55%; Accuracy Avg: 75.62%;
# Epoch: 189; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.34%; Accuracy Avg: 75.69%;
# Epoch: 190; D_loss: 0.21; G_loss: 0.26; Accuracy: 74.14%; Accuracy Avg: 75.59%;
# Epoch: 191; D_loss: 0.21; G_loss: 0.26; Accuracy: 73.87%; Accuracy Avg: 75.52%;
# Epoch: 192; D_loss: 0.20; G_loss: 0.26; Accuracy: 74.08%; Accuracy Avg: 75.52%;
# Epoch: 193; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.16%; Accuracy Avg: 75.55%;
# Epoch: 194; D_loss: 0.20; G_loss: 0.26; Accuracy: 74.19%; Accuracy Avg: 75.66%;
# Epoch: 195; D_loss: 0.20; G_loss: 0.26; Accuracy: 75.00%; Accuracy Avg: 75.66%;
# Epoch: 196; D_loss: 0.20; G_loss: 0.26; Accuracy: 74.50%; Accuracy Avg: 75.72%;
# Epoch: 197; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.41%; Accuracy Avg: 75.65%;
# Epoch: 198; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.22%; Accuracy Avg: 75.83%;
# Epoch: 199; D_loss: 0.20; G_loss: 0.26; Accuracy: 74.48%; Accuracy Avg: 75.81%;
# Epoch: 200; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.26%; Accuracy Avg: 75.95%;
# Epoch: 201; D_loss: 0.20; G_loss: 0.25; Accuracy: 76.02%; Accuracy Avg: 75.73%;
# Epoch: 202; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.52%; Accuracy Avg: 75.90%;
# Epoch: 203; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.59%; Accuracy Avg: 75.86%;
# Epoch: 204; D_loss: 0.20; G_loss: 0.26; Accuracy: 73.97%; Accuracy Avg: 76.03%;
# Epoch: 205; D_loss: 0.20; G_loss: 0.25; Accuracy: 74.65%; Accuracy Avg: 76.12%;
# Epoch: 206; D_loss: 0.20; G_loss: 0.25; Accuracy: 73.56%; Accuracy Avg: 76.12%;
# Epoch: 207; D_loss: 0.20; G_loss: 0.25; Accuracy: 75.02%; Accuracy Avg: 76.18%;
# Epoch: 208; D_loss: 0.20; G_loss: 0.25; Accuracy: 75.32%; Accuracy Avg: 76.26%;
# Epoch: 209; D_loss: 0.19; G_loss: 0.26; Accuracy: 74.21%; Accuracy Avg: 76.34%;
# Epoch: 210; D_loss: 0.20; G_loss: 0.25; Accuracy: 74.31%; Accuracy Avg: 76.37%;
# Epoch: 211; D_loss: 0.20; G_loss: 0.25; Accuracy: 74.88%; Accuracy Avg: 76.51%;
# Epoch: 212; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.72%; Accuracy Avg: 76.49%;
# Epoch: 213; D_loss: 0.19; G_loss: 0.25; Accuracy: 74.41%; Accuracy Avg: 76.52%;
# Epoch: 214; D_loss: 0.19; G_loss: 0.26; Accuracy: 74.30%; Accuracy Avg: 76.46%;
# Epoch: 215; D_loss: 0.20; G_loss: 0.25; Accuracy: 75.78%; Accuracy Avg: 76.43%;
# Epoch: 216; D_loss: 0.19; G_loss: 0.25; Accuracy: 72.79%; Accuracy Avg: 76.45%;
# Epoch: 217; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.43%; Accuracy Avg: 76.37%;
# Epoch: 218; D_loss: 0.20; G_loss: 0.25; Accuracy: 75.38%; Accuracy Avg: 76.43%;
# Epoch: 219; D_loss: 0.20; G_loss: 0.25; Accuracy: 75.12%; Accuracy Avg: 76.43%;
# Epoch: 220; D_loss: 0.19; G_loss: 0.25; Accuracy: 74.82%; Accuracy Avg: 76.64%;
# Epoch: 221; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.56%; Accuracy Avg: 76.77%;
# Epoch: 222; D_loss: 0.19; G_loss: 0.25; Accuracy: 76.36%; Accuracy Avg: 76.79%;
# Epoch: 223; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.81%; Accuracy Avg: 76.97%;
# Epoch: 224; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.91%; Accuracy Avg: 76.99%;
# Epoch: 225; D_loss: 0.19; G_loss: 0.25; Accuracy: 76.73%; Accuracy Avg: 77.01%;
# Epoch: 226; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.12%; Accuracy Avg: 77.04%;
# Epoch: 227; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.40%; Accuracy Avg: 77.10%;
# Epoch: 228; D_loss: 0.18; G_loss: 0.25; Accuracy: 74.80%; Accuracy Avg: 77.16%;
# Epoch: 229; D_loss: 0.19; G_loss: 0.25; Accuracy: 75.99%; Accuracy Avg: 77.20%;
# Epoch: 230; D_loss: 0.19; G_loss: 0.25; Accuracy: 76.24%; Accuracy Avg: 77.26%;
# Epoch: 231; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.06%; Accuracy Avg: 77.28%;
# Epoch: 232; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.95%; Accuracy Avg: 77.22%;
# Epoch: 233; D_loss: 0.19; G_loss: 0.25; Accuracy: 76.71%; Accuracy Avg: 77.38%;
# Epoch: 234; D_loss: 0.18; G_loss: 0.25; Accuracy: 76.38%; Accuracy Avg: 77.37%;
# Epoch: 235; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.00%; Accuracy Avg: 77.40%;
# Epoch: 236; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.99%; Accuracy Avg: 77.46%;
# Epoch: 237; D_loss: 0.18; G_loss: 0.25; Accuracy: 76.74%; Accuracy Avg: 77.59%;
# Epoch: 238; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.10%; Accuracy Avg: 77.70%;
# Epoch: 239; D_loss: 0.18; G_loss: 0.25; Accuracy: 76.01%; Accuracy Avg: 77.68%;
# Epoch: 240; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.92%; Accuracy Avg: 77.77%;
# Epoch: 241; D_loss: 0.18; G_loss: 0.25; Accuracy: 75.99%; Accuracy Avg: 77.70%;
# Epoch: 242; D_loss: 0.18; G_loss: 0.24; Accuracy: 76.45%; Accuracy Avg: 77.69%;
# Epoch: 243; D_loss: 0.18; G_loss: 0.25; Accuracy: 76.06%; Accuracy Avg: 77.73%;
# Epoch: 244; D_loss: 0.18; G_loss: 0.25; Accuracy: 77.08%; Accuracy Avg: 77.73%;
# Epoch: 245; D_loss: 0.18; G_loss: 0.25; Accuracy: 77.39%; Accuracy Avg: 77.81%;
# Epoch: 246; D_loss: 0.18; G_loss: 0.24; Accuracy: 77.35%; Accuracy Avg: 77.79%;
# Epoch: 247; D_loss: 0.18; G_loss: 0.25; Accuracy: 76.54%; Accuracy Avg: 77.83%;
# Epoch: 248; D_loss: 0.17; G_loss: 0.24; Accuracy: 75.91%; Accuracy Avg: 77.88%;
# Epoch: 249; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.73%; Accuracy Avg: 77.87%;
# Epoch: 250; D_loss: 0.18; G_loss: 0.24; Accuracy: 75.78%; Accuracy Avg: 77.83%;
# Epoch: 251; D_loss: 0.17; G_loss: 0.25; Accuracy: 76.59%; Accuracy Avg: 77.91%;
# Epoch: 252; D_loss: 0.17; G_loss: 0.25; Accuracy: 77.13%; Accuracy Avg: 78.06%;
# Epoch: 253; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.00%; Accuracy Avg: 78.10%;
# Epoch: 254; D_loss: 0.17; G_loss: 0.24; Accuracy: 75.68%; Accuracy Avg: 78.33%;
# Epoch: 255; D_loss: 0.18; G_loss: 0.24; Accuracy: 75.32%; Accuracy Avg: 78.22%;
# Epoch: 256; D_loss: 0.18; G_loss: 0.24; Accuracy: 77.54%; Accuracy Avg: 78.25%;
# Epoch: 257; D_loss: 0.17; G_loss: 0.25; Accuracy: 76.55%; Accuracy Avg: 78.25%;
# Epoch: 258; D_loss: 0.18; G_loss: 0.24; Accuracy: 76.17%; Accuracy Avg: 78.30%;
# Epoch: 259; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.06%; Accuracy Avg: 78.21%;
# Epoch: 260; D_loss: 0.17; G_loss: 0.24; Accuracy: 78.06%; Accuracy Avg: 78.41%;
# Epoch: 261; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.86%; Accuracy Avg: 78.38%;
# Epoch: 262; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.08%; Accuracy Avg: 78.45%;
# Epoch: 263; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.43%; Accuracy Avg: 78.51%;
# Epoch: 264; D_loss: 0.17; G_loss: 0.24; Accuracy: 75.76%; Accuracy Avg: 78.48%;
# Epoch: 265; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.54%; Accuracy Avg: 78.46%;
# Epoch: 266; D_loss: 0.18; G_loss: 0.24; Accuracy: 76.11%; Accuracy Avg: 78.41%;
# Epoch: 267; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.04%; Accuracy Avg: 78.33%;
# Epoch: 268; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.62%; Accuracy Avg: 78.34%;
# Epoch: 269; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.65%; Accuracy Avg: 78.36%;
# Epoch: 270; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.33%; Accuracy Avg: 78.38%;
# Epoch: 271; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.78%; Accuracy Avg: 78.33%;
# Epoch: 272; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.62%; Accuracy Avg: 78.41%;
# Epoch: 273; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.21%; Accuracy Avg: 78.49%;
# Epoch: 274; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.14%; Accuracy Avg: 78.52%;
# Epoch: 275; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.42%; Accuracy Avg: 78.69%;
# Epoch: 276; D_loss: 0.17; G_loss: 0.24; Accuracy: 78.02%; Accuracy Avg: 78.80%;
# Epoch: 277; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.34%; Accuracy Avg: 78.72%;
# Epoch: 278; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.51%; Accuracy Avg: 78.73%;
# Epoch: 279; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.47%; Accuracy Avg: 78.87%;
# Epoch: 280; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.98%; Accuracy Avg: 78.95%;
# Epoch: 281; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.01%; Accuracy Avg: 78.89%;
# Epoch: 282; D_loss: 0.16; G_loss: 0.24; Accuracy: 75.17%; Accuracy Avg: 78.88%;
# Epoch: 283; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.34%; Accuracy Avg: 78.67%;
# Epoch: 284; D_loss: 0.17; G_loss: 0.24; Accuracy: 76.84%; Accuracy Avg: 78.67%;
# Epoch: 285; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.67%; Accuracy Avg: 78.53%;
# Epoch: 286; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.34%; Accuracy Avg: 78.60%;
# Epoch: 287; D_loss: 0.16; G_loss: 0.24; Accuracy: 76.26%; Accuracy Avg: 78.69%;
# Epoch: 288; D_loss: 0.17; G_loss: 0.24; Accuracy: 77.46%; Accuracy Avg: 78.73%;
# Epoch: 289; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.89%; Accuracy Avg: 78.81%;
# Epoch: 290; D_loss: 0.16; G_loss: 0.24; Accuracy: 76.70%; Accuracy Avg: 78.90%;
# Epoch: 291; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.21%; Accuracy Avg: 79.00%;
# Epoch: 292; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.70%; Accuracy Avg: 79.03%;
# Epoch: 293; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.81%; Accuracy Avg: 78.97%;
# Epoch: 294; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.73%; Accuracy Avg: 78.93%;
# Epoch: 295; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.81%; Accuracy Avg: 79.04%;
# Epoch: 296; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.18%; Accuracy Avg: 79.07%;
# Epoch: 297; D_loss: 0.16; G_loss: 0.23; Accuracy: 77.34%; Accuracy Avg: 79.01%;
# Epoch: 298; D_loss: 0.16; G_loss: 0.24; Accuracy: 76.90%; Accuracy Avg: 78.89%;
# Epoch: 299; D_loss: 0.16; G_loss: 0.24; Accuracy: 76.68%; Accuracy Avg: 78.90%;
# Epoch: 300; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.14%; Accuracy Avg: 78.93%;
# Epoch: 301; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.66%; Accuracy Avg: 78.95%;
# Epoch: 302; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.53%; Accuracy Avg: 79.03%;
# Epoch: 303; D_loss: 0.16; G_loss: 0.24; Accuracy: 78.06%; Accuracy Avg: 79.17%;
# Epoch: 304; D_loss: 0.16; G_loss: 0.24; Accuracy: 76.61%; Accuracy Avg: 79.07%;
# Epoch: 305; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.35%; Accuracy Avg: 79.06%;
# Epoch: 306; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.56%; Accuracy Avg: 79.08%;
# Epoch: 307; D_loss: 0.16; G_loss: 0.23; Accuracy: 77.94%; Accuracy Avg: 79.18%;
# Epoch: 308; D_loss: 0.16; G_loss: 0.23; Accuracy: 77.79%; Accuracy Avg: 79.30%;
# Epoch: 309; D_loss: 0.16; G_loss: 0.23; Accuracy: 78.07%; Accuracy Avg: 79.32%;
# Epoch: 310; D_loss: 0.16; G_loss: 0.23; Accuracy: 76.64%; Accuracy Avg: 79.36%;
# Epoch: 311; D_loss: 0.16; G_loss: 0.23; Accuracy: 76.38%; Accuracy Avg: 79.29%;
# Epoch: 312; D_loss: 0.15; G_loss: 0.24; Accuracy: 77.88%; Accuracy Avg: 79.27%;
# Epoch: 313; D_loss: 0.16; G_loss: 0.23; Accuracy: 76.94%; Accuracy Avg: 79.30%;
# Epoch: 314; D_loss: 0.15; G_loss: 0.24; Accuracy: 76.91%; Accuracy Avg: 79.32%;
# Epoch: 315; D_loss: 0.16; G_loss: 0.23; Accuracy: 76.61%; Accuracy Avg: 79.42%;
# Epoch: 316; D_loss: 0.16; G_loss: 0.23; Accuracy: 77.10%; Accuracy Avg: 79.38%;
# Epoch: 317; D_loss: 0.16; G_loss: 0.23; Accuracy: 77.64%; Accuracy Avg: 79.30%;
# Epoch: 318; D_loss: 0.15; G_loss: 0.24; Accuracy: 77.95%; Accuracy Avg: 79.35%;
# Epoch: 319; D_loss: 0.15; G_loss: 0.24; Accuracy: 78.78%; Accuracy Avg: 79.37%;
# Epoch: 320; D_loss: 0.15; G_loss: 0.24; Accuracy: 77.47%; Accuracy Avg: 79.39%;
# Epoch: 321; D_loss: 0.15; G_loss: 0.24; Accuracy: 77.79%; Accuracy Avg: 79.55%;
# Epoch: 322; D_loss: 0.16; G_loss: 0.24; Accuracy: 77.73%; Accuracy Avg: 79.61%;
# Epoch: 323; D_loss: 0.15; G_loss: 0.24; Accuracy: 79.16%; Accuracy Avg: 79.60%;
# Epoch: 324; D_loss: 0.16; G_loss: 0.23; Accuracy: 78.00%; Accuracy Avg: 79.57%;
# Epoch: 325; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.14%; Accuracy Avg: 79.64%;
# Epoch: 326; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.78%; Accuracy Avg: 79.73%;
# Epoch: 327; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.24%; Accuracy Avg: 79.85%;
# Epoch: 328; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.96%; Accuracy Avg: 79.94%;
# Epoch: 329; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.32%; Accuracy Avg: 79.91%;
# Epoch: 330; D_loss: 0.16; G_loss: 0.23; Accuracy: 78.37%; Accuracy Avg: 79.85%;
# Epoch: 331; D_loss: 0.15; G_loss: 0.23; Accuracy: 77.81%; Accuracy Avg: 79.90%;
# Epoch: 332; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.29%; Accuracy Avg: 79.91%;
# Epoch: 333; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.06%; Accuracy Avg: 79.90%;
# Epoch: 334; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.83%; Accuracy Avg: 79.96%;
# Epoch: 335; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.03%; Accuracy Avg: 80.07%;
# Epoch: 336; D_loss: 0.15; G_loss: 0.23; Accuracy: 77.58%; Accuracy Avg: 80.08%;
# Epoch: 337; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.31%; Accuracy Avg: 79.98%;
# Epoch: 338; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.79%; Accuracy Avg: 79.93%;
# Epoch: 339; D_loss: 0.15; G_loss: 0.24; Accuracy: 78.48%; Accuracy Avg: 80.01%;
# Epoch: 340; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.16%; Accuracy Avg: 80.06%;
# Epoch: 341; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.68%; Accuracy Avg: 80.13%;
# Epoch: 342; D_loss: 0.15; G_loss: 0.23; Accuracy: 77.66%; Accuracy Avg: 80.14%;
# Epoch: 343; D_loss: 0.15; G_loss: 0.23; Accuracy: 77.44%; Accuracy Avg: 80.17%;
# Epoch: 344; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.02%; Accuracy Avg: 80.11%;
# Epoch: 345; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.58%; Accuracy Avg: 80.08%;
# Epoch: 346; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.16%; Accuracy Avg: 80.13%;
# Epoch: 347; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.80%; Accuracy Avg: 80.08%;
# Epoch: 348; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.74%; Accuracy Avg: 80.08%;
# Epoch: 349; D_loss: 0.15; G_loss: 0.23; Accuracy: 77.95%; Accuracy Avg: 80.11%;
# Epoch: 350; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.33%; Accuracy Avg: 80.03%;
# Epoch: 351; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.72%; Accuracy Avg: 80.05%;
# Epoch: 352; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.17%; Accuracy Avg: 80.08%;
# Epoch: 353; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.55%; Accuracy Avg: 80.02%;
# Epoch: 354; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.52%; Accuracy Avg: 80.08%;
# Epoch: 355; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.77%; Accuracy Avg: 80.11%;
# Epoch: 356; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.57%; Accuracy Avg: 80.15%;
# Epoch: 357; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.24%; Accuracy Avg: 80.18%;
# Epoch: 358; D_loss: 0.15; G_loss: 0.23; Accuracy: 79.99%; Accuracy Avg: 80.22%;
# Epoch: 359; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.78%; Accuracy Avg: 80.34%;
# Epoch: 360; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.38%; Accuracy Avg: 80.41%;
# Epoch: 361; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.36%; Accuracy Avg: 80.42%;
# Epoch: 362; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.41%; Accuracy Avg: 80.38%;
# Epoch: 363; D_loss: 0.14; G_loss: 0.23; Accuracy: 77.55%; Accuracy Avg: 80.29%;
# Epoch: 364; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.67%; Accuracy Avg: 80.31%;
# Epoch: 365; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.87%; Accuracy Avg: 80.39%;
# Epoch: 366; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.47%; Accuracy Avg: 80.34%;
# Epoch: 367; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.45%; Accuracy Avg: 80.40%;
# Epoch: 368; D_loss: 0.15; G_loss: 0.23; Accuracy: 78.66%; Accuracy Avg: 80.35%;
# Epoch: 369; D_loss: 0.15; G_loss: 0.23; Accuracy: 77.62%; Accuracy Avg: 80.37%;
# Epoch: 370; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.15%; Accuracy Avg: 80.35%;
# Epoch: 371; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.92%; Accuracy Avg: 80.42%;
# Epoch: 372; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.02%; Accuracy Avg: 80.55%;
# Epoch: 373; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.45%; Accuracy Avg: 80.49%;
# Epoch: 374; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.39%; Accuracy Avg: 80.59%;
# Epoch: 375; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.25%; Accuracy Avg: 80.60%;
# Epoch: 376; D_loss: 0.15; G_loss: 0.22; Accuracy: 79.76%; Accuracy Avg: 80.61%;
# Epoch: 377; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.84%; Accuracy Avg: 80.58%;
# Epoch: 378; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.51%; Accuracy Avg: 80.63%;
# Epoch: 379; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.55%; Accuracy Avg: 80.58%;
# Epoch: 380; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.92%; Accuracy Avg: 80.62%;
# Epoch: 381; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.77%; Accuracy Avg: 80.49%;
# Epoch: 382; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.61%; Accuracy Avg: 80.51%;
# Epoch: 383; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.56%; Accuracy Avg: 80.47%;
# Epoch: 384; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.58%; Accuracy Avg: 80.49%;
# Epoch: 385; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.11%; Accuracy Avg: 80.49%;
# Epoch: 386; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.73%; Accuracy Avg: 80.43%;
# Epoch: 387; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.66%; Accuracy Avg: 80.44%;
# Epoch: 388; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.92%; Accuracy Avg: 80.54%;
# Epoch: 389; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.30%; Accuracy Avg: 80.59%;
# Epoch: 390; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.13%; Accuracy Avg: 80.59%;
# Epoch: 391; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.04%; Accuracy Avg: 80.64%;
# Epoch: 392; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.77%; Accuracy Avg: 80.72%;
# Epoch: 393; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.74%; Accuracy Avg: 80.63%;
# Epoch: 394; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.01%; Accuracy Avg: 80.64%;
# Epoch: 395; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.68%; Accuracy Avg: 80.56%;
# Epoch: 396; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.33%; Accuracy Avg: 80.66%;
# Epoch: 397; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.54%; Accuracy Avg: 80.66%;
# Epoch: 398; D_loss: 0.14; G_loss: 0.23; Accuracy: 80.01%; Accuracy Avg: 80.61%;
# Epoch: 399; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.12%; Accuracy Avg: 80.61%;
# Epoch: 400; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.03%; Accuracy Avg: 80.65%;
# Epoch: 401; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.54%; Accuracy Avg: 80.68%;
# Epoch: 402; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.56%; Accuracy Avg: 80.72%;
# Epoch: 403; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.61%; Accuracy Avg: 80.73%;
# Epoch: 404; D_loss: 0.14; G_loss: 0.23; Accuracy: 80.14%; Accuracy Avg: 80.80%;
# Epoch: 405; D_loss: 0.14; G_loss: 0.23; Accuracy: 78.87%; Accuracy Avg: 80.73%;
# Epoch: 406; D_loss: 0.14; G_loss: 0.23; Accuracy: 80.44%; Accuracy Avg: 80.70%;
# Epoch: 407; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.03%; Accuracy Avg: 80.81%;
# Epoch: 408; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.08%; Accuracy Avg: 80.75%;
# Epoch: 409; D_loss: 0.14; G_loss: 0.22; Accuracy: 80.28%; Accuracy Avg: 80.81%;
# Epoch: 410; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.39%; Accuracy Avg: 80.89%;
# Epoch: 411; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.10%; Accuracy Avg: 80.92%;
# Epoch: 412; D_loss: 0.14; G_loss: 0.22; Accuracy: 79.76%; Accuracy Avg: 80.98%;
# Epoch: 413; D_loss: 0.14; G_loss: 0.22; Accuracy: 79.22%; Accuracy Avg: 81.01%;
# Epoch: 414; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.46%; Accuracy Avg: 80.98%;
# Epoch: 415; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.29%; Accuracy Avg: 81.03%;
# Epoch: 416; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.23%; Accuracy Avg: 81.03%;
# Epoch: 417; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.65%; Accuracy Avg: 81.04%;
# Epoch: 418; D_loss: 0.14; G_loss: 0.23; Accuracy: 79.30%; Accuracy Avg: 81.07%;
# Epoch: 419; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.75%; Accuracy Avg: 81.08%;
# Epoch: 420; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.38%; Accuracy Avg: 81.15%;
# Epoch: 421; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.07%; Accuracy Avg: 80.98%;
# Epoch: 422; D_loss: 0.13; G_loss: 0.23; Accuracy: 78.91%; Accuracy Avg: 80.83%;
# Epoch: 423; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.00%; Accuracy Avg: 80.69%;
# Epoch: 424; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.76%; Accuracy Avg: 80.70%;
# Epoch: 425; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.84%; Accuracy Avg: 80.81%;
# Epoch: 426; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.31%; Accuracy Avg: 80.81%;
# Epoch: 427; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.78%; Accuracy Avg: 80.76%;
# Epoch: 428; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.74%; Accuracy Avg: 80.74%;
# Epoch: 429; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.80%; Accuracy Avg: 80.73%;
# Epoch: 430; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.11%; Accuracy Avg: 80.80%;
# Epoch: 431; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.52%; Accuracy Avg: 80.86%;
# Epoch: 432; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.18%; Accuracy Avg: 80.91%;
# Epoch: 433; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.86%; Accuracy Avg: 80.94%;
# Epoch: 434; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.95%; Accuracy Avg: 80.90%;
# Epoch: 435; D_loss: 0.13; G_loss: 0.23; Accuracy: 78.95%; Accuracy Avg: 81.01%;
# Epoch: 436; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.81%; Accuracy Avg: 81.00%;
# Epoch: 437; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.27%; Accuracy Avg: 81.07%;
# Epoch: 438; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.41%; Accuracy Avg: 80.99%;
# Epoch: 439; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.95%; Accuracy Avg: 80.94%;
# Epoch: 440; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.66%; Accuracy Avg: 81.09%;
# Epoch: 441; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.36%; Accuracy Avg: 81.18%;
# Epoch: 442; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.25%; Accuracy Avg: 81.19%;
# Epoch: 443; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.06%; Accuracy Avg: 81.28%;
# Epoch: 444; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.90%; Accuracy Avg: 81.22%;
# Epoch: 445; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.09%; Accuracy Avg: 81.42%;
# Epoch: 446; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.11%; Accuracy Avg: 81.42%;
# Epoch: 447; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.73%; Accuracy Avg: 81.29%;
# Epoch: 448; D_loss: 0.12; G_loss: 0.22; Accuracy: 79.69%; Accuracy Avg: 81.23%;
# Epoch: 449; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.91%; Accuracy Avg: 81.24%;
# Epoch: 450; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.76%; Accuracy Avg: 81.13%;
# Epoch: 451; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.50%; Accuracy Avg: 81.18%;
# Epoch: 452; D_loss: 0.13; G_loss: 0.22; Accuracy: 80.19%; Accuracy Avg: 81.23%;
# Epoch: 453; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.68%; Accuracy Avg: 81.28%;
# Epoch: 454; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.61%; Accuracy Avg: 81.26%;
# Epoch: 455; D_loss: 0.13; G_loss: 0.22; Accuracy: 80.21%; Accuracy Avg: 81.11%;
# Epoch: 456; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.81%; Accuracy Avg: 81.08%;
# Epoch: 457; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.57%; Accuracy Avg: 81.09%;
# Epoch: 458; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.65%; Accuracy Avg: 81.11%;
# Epoch: 459; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.29%; Accuracy Avg: 81.16%;
# Epoch: 460; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.65%; Accuracy Avg: 81.11%;
# Epoch: 461; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.15%; Accuracy Avg: 81.13%;
# Epoch: 462; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.91%; Accuracy Avg: 81.15%;
# Epoch: 463; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.61%; Accuracy Avg: 81.08%;
# Epoch: 464; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.94%; Accuracy Avg: 81.15%;
# Epoch: 465; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.34%; Accuracy Avg: 81.15%;
# Epoch: 466; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.73%; Accuracy Avg: 81.15%;
# Epoch: 467; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.44%; Accuracy Avg: 81.14%;
# Epoch: 468; D_loss: 0.13; G_loss: 0.23; Accuracy: 80.01%; Accuracy Avg: 81.18%;
# Epoch: 469; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.90%; Accuracy Avg: 81.09%;
# Epoch: 470; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.84%; Accuracy Avg: 81.19%;
# Epoch: 471; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.12%; Accuracy Avg: 81.19%;
# Epoch: 472; D_loss: 0.13; G_loss: 0.23; Accuracy: 79.85%; Accuracy Avg: 81.27%;
# Epoch: 473; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.48%; Accuracy Avg: 81.17%;
# Epoch: 474; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.10%; Accuracy Avg: 81.16%;
# Epoch: 475; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.43%; Accuracy Avg: 81.14%;
# Epoch: 476; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.82%; Accuracy Avg: 81.01%;
# Epoch: 477; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.97%; Accuracy Avg: 81.04%;
# Epoch: 478; D_loss: 0.13; G_loss: 0.22; Accuracy: 79.84%; Accuracy Avg: 81.02%;
# Epoch: 479; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.89%; Accuracy Avg: 80.87%;
# Epoch: 480; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.35%; Accuracy Avg: 80.92%;
# Epoch: 481; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.29%; Accuracy Avg: 81.05%;
# Epoch: 482; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.19%; Accuracy Avg: 81.21%;
# Epoch: 483; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.07%; Accuracy Avg: 81.11%;
# Epoch: 484; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.96%; Accuracy Avg: 81.15%;
# Epoch: 485; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.23%; Accuracy Avg: 81.16%;
# Epoch: 486; D_loss: 0.13; G_loss: 0.22; Accuracy: 80.38%; Accuracy Avg: 81.28%;
# Epoch: 487; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.23%; Accuracy Avg: 81.33%;
# Epoch: 488; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.73%; Accuracy Avg: 81.37%;
# Epoch: 489; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.15%; Accuracy Avg: 81.54%;
# Epoch: 490; D_loss: 0.12; G_loss: 0.23; Accuracy: 79.59%; Accuracy Avg: 81.46%;
# Epoch: 491; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.07%; Accuracy Avg: 81.48%;
# Epoch: 492; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.37%; Accuracy Avg: 81.40%;
# Epoch: 493; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.74%; Accuracy Avg: 81.40%;
# Epoch: 494; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.98%; Accuracy Avg: 81.40%;
# Epoch: 495; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.42%; Accuracy Avg: 81.38%;
# Epoch: 496; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.37%; Accuracy Avg: 81.33%;
# Epoch: 497; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.26%; Accuracy Avg: 81.43%;
# Epoch: 498; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.38%; Accuracy Avg: 81.34%;
# Epoch: 499; D_loss: 0.12; G_loss: 0.23; Accuracy: 80.14%; Accuracy Avg: 81.33%;
# Epoch: 500; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.20%; Accuracy Avg: 81.44%;
# Epoch: 501; D_loss: 0.12; G_loss: 0.22; Accuracy: 79.89%; Accuracy Avg: 81.44%;
# Epoch: 502; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.78%; Accuracy Avg: 81.33%;
# Epoch: 503; D_loss: 0.12; G_loss: 0.22; Accuracy: 80.62%; Accuracy Avg: 81.34%;

In [ ]:
# original tf code
# Epoch 0 | time = 138s | loss gen = 0.3197 | loss lab = 0.4926 | loss unl = 1.7710 | train acc = 0.3508| test acc = 0.4540 | test acc ema = 0.1240
# Epoch 1 | time = 137s | loss gen = 0.3553 | loss lab = 0.4076 | loss unl = 1.1665 | train acc = 0.5815| test acc = 0.4974 | test acc ema = 0.1931
# Epoch 2 | time = 138s | loss gen = 0.4759 | loss lab = 0.3645 | loss unl = 0.7594 | train acc = 0.7296| test acc = 0.5298 | test acc ema = 0.2674
# Epoch 3 | time = 137s | loss gen = 0.5666 | loss lab = 0.3494 | loss unl = 0.4881 | train acc = 0.8292| test acc = 0.5579 | test acc ema = 0.3614
# Epoch 4 | time = 137s | loss gen = 0.6288 | loss lab = 0.3303 | loss unl = 0.3247 | train acc = 0.8909| test acc = 0.5394 | test acc ema = 0.4546
# Epoch 5 | time = 138s | loss gen = 0.6424 | loss lab = 0.3192 | loss unl = 0.2340 | train acc = 0.9257| test acc = 0.5693 | test acc ema = 0.5091
# Epoch 6 | time = 138s | loss gen = 0.6477 | loss lab = 0.3022 | loss unl = 0.1845 | train acc = 0.9427| test acc = 0.5544 | test acc ema = 0.5492
# Epoch 7 | time = 137s | loss gen = 0.6481 | loss lab = 0.2938 | loss unl = 0.1560 | train acc = 0.9543| test acc = 0.5695 | test acc ema = 0.5781
# Epoch 8 | time = 137s | loss gen = 0.6318 | loss lab = 0.2896 | loss unl = 0.1327 | train acc = 0.9626| test acc = 0.5777 | test acc ema = 0.5932
# Epoch 9 | time = 137s | loss gen = 0.6352 | loss lab = 0.2733 | loss unl = 0.1131 | train acc = 0.9705| test acc = 0.5743 | test acc ema = 0.5996
# Epoch 10 | time = 137s | loss gen = 0.6145 | loss lab = 0.2776 | loss unl = 0.1021 | train acc = 0.9739| test acc = 0.5858 | test acc ema = 0.6067
# Epoch 11 | time = 137s | loss gen = 0.5971 | loss lab = 0.2733 | loss unl = 0.0922 | train acc = 0.9771| test acc = 0.5985 | test acc ema = 0.6092
# Epoch 12 | time = 137s | loss gen = 0.5782 | loss lab = 0.2713 | loss unl = 0.0844 | train acc = 0.9800| test acc = 0.6055 | test acc ema = 0.6122
# Epoch 13 | time = 137s | loss gen = 0.5672 | loss lab = 0.2727 | loss unl = 0.0774 | train acc = 0.9828| test acc = 0.6090 | test acc ema = 0.6161
# Epoch 14 | time = 137s | loss gen = 0.5598 | loss lab = 0.2638 | loss unl = 0.0701 | train acc = 0.9851| test acc = 0.6182 | test acc ema = 0.6182
# Epoch 15 | time = 137s | loss gen = 0.5439 | loss lab = 0.2668 | loss unl = 0.0663 | train acc = 0.9860| test acc = 0.6101 | test acc ema = 0.6203
# Epoch 16 | time = 137s | loss gen = 0.5278 | loss lab = 0.2693 | loss unl = 0.0617 | train acc = 0.9878| test acc = 0.6253 | test acc ema = 0.6230
# Epoch 17 | time = 137s | loss gen = 0.5142 | loss lab = 0.2694 | loss unl = 0.0580 | train acc = 0.9892| test acc = 0.6203 | test acc ema = 0.6243
# Epoch 18 | time = 137s | loss gen = 0.5070 | loss lab = 0.2679 | loss unl = 0.0555 | train acc = 0.9894| test acc = 0.6053 | test acc ema = 0.6301
# Epoch 19 | time = 137s | loss gen = 0.4994 | loss lab = 0.2672 | loss unl = 0.0525 | train acc = 0.9907| test acc = 0.6133 | test acc ema = 0.6325
# Epoch 20 | time = 137s | loss gen = 0.4918 | loss lab = 0.2651 | loss unl = 0.0501 | train acc = 0.9914| test acc = 0.6287 | test acc ema = 0.6383
# Epoch 21 | time = 137s | loss gen = 0.4840 | loss lab = 0.2608 | loss unl = 0.0470 | train acc = 0.9916| test acc = 0.5989 | test acc ema = 0.6395
# Epoch 22 | time = 137s | loss gen = 0.4752 | loss lab = 0.2611 | loss unl = 0.0457 | train acc = 0.9921| test acc = 0.6396 | test acc ema = 0.6453
# Epoch 23 | time = 136s | loss gen = 0.4677 | loss lab = 0.2597 | loss unl = 0.0432 | train acc = 0.9928| test acc = 0.6550 | test acc ema = 0.6507
# Epoch 24 | time = 137s | loss gen = 0.4546 | loss lab = 0.2665 | loss unl = 0.0429 | train acc = 0.9929| test acc = 0.6589 | test acc ema = 0.6554
# Epoch 25 | time = 137s | loss gen = 0.4506 | loss lab = 0.2612 | loss unl = 0.0406 | train acc = 0.9937| test acc = 0.6533 | test acc ema = 0.6616
# Epoch 26 | time = 137s | loss gen = 0.4457 | loss lab = 0.2569 | loss unl = 0.0381 | train acc = 0.9944| test acc = 0.6693 | test acc ema = 0.6639
# Epoch 27 | time = 137s | loss gen = 0.4431 | loss lab = 0.2543 | loss unl = 0.0371 | train acc = 0.9943| test acc = 0.6406 | test acc ema = 0.6672
# Epoch 28 | time = 137s | loss gen = 0.4332 | loss lab = 0.2548 | loss unl = 0.0361 | train acc = 0.9942| test acc = 0.6710 | test acc ema = 0.6716
# Epoch 29 | time = 136s | loss gen = 0.4289 | loss lab = 0.2549 | loss unl = 0.0357 | train acc = 0.9945| test acc = 0.6716 | test acc ema = 0.6754
# Epoch 30 | time = 137s | loss gen = 0.4246 | loss lab = 0.2548 | loss unl = 0.0341 | train acc = 0.9952| test acc = 0.6707 | test acc ema = 0.6792
# Epoch 31 | time = 137s | loss gen = 0.4196 | loss lab = 0.2490 | loss unl = 0.0331 | train acc = 0.9951| test acc = 0.6568 | test acc ema = 0.6831
# Epoch 32 | time = 137s | loss gen = 0.4169 | loss lab = 0.2471 | loss unl = 0.0326 | train acc = 0.9952| test acc = 0.6808 | test acc ema = 0.6859
# Epoch 33 | time = 137s | loss gen = 0.4065 | loss lab = 0.2510 | loss unl = 0.0320 | train acc = 0.9954| test acc = 0.6802 | test acc ema = 0.6892
# Epoch 34 | time = 137s | loss gen = 0.4047 | loss lab = 0.2490 | loss unl = 0.0308 | train acc = 0.9959| test acc = 0.6919 | test acc ema = 0.6933
# Epoch 35 | time = 137s | loss gen = 0.3974 | loss lab = 0.2491 | loss unl = 0.0303 | train acc = 0.9957| test acc = 0.6684 | test acc ema = 0.6965
# Epoch 36 | time = 137s | loss gen = 0.3955 | loss lab = 0.2434 | loss unl = 0.0288 | train acc = 0.9962| test acc = 0.6946 | test acc ema = 0.6978
# Epoch 37 | time = 137s | loss gen = 0.3938 | loss lab = 0.2391 | loss unl = 0.0278 | train acc = 0.9963| test acc = 0.6879 | test acc ema = 0.7000
# Epoch 38 | time = 137s | loss gen = 0.3904 | loss lab = 0.2378 | loss unl = 0.0268 | train acc = 0.9963| test acc = 0.7103 | test acc ema = 0.7021
# Epoch 39 | time = 136s | loss gen = 0.3844 | loss lab = 0.2379 | loss unl = 0.0261 | train acc = 0.9968| test acc = 0.6907 | test acc ema = 0.7060
# Epoch 40 | time = 137s | loss gen = 0.3766 | loss lab = 0.2396 | loss unl = 0.0262 | train acc = 0.9961| test acc = 0.7192 | test acc ema = 0.7119
# Epoch 41 | time = 137s | loss gen = 0.3768 | loss lab = 0.2341 | loss unl = 0.0261 | train acc = 0.9963| test acc = 0.7060 | test acc ema = 0.7148
# Epoch 42 | time = 137s | loss gen = 0.3737 | loss lab = 0.2333 | loss unl = 0.0245 | train acc = 0.9969| test acc = 0.7002 | test acc ema = 0.7150
# Epoch 43 | time = 136s | loss gen = 0.3679 | loss lab = 0.2322 | loss unl = 0.0241 | train acc = 0.9973| test acc = 0.6934 | test acc ema = 0.7186
# Epoch 44 | time = 136s | loss gen = 0.3675 | loss lab = 0.2341 | loss unl = 0.0249 | train acc = 0.9966| test acc = 0.7013 | test acc ema = 0.7189
# Epoch 45 | time = 136s | loss gen = 0.3654 | loss lab = 0.2301 | loss unl = 0.0226 | train acc = 0.9972| test acc = 0.7201 | test acc ema = 0.7215
# Epoch 46 | time = 136s | loss gen = 0.3590 | loss lab = 0.2309 | loss unl = 0.0223 | train acc = 0.9978| test acc = 0.6996 | test acc ema = 0.7239
# Epoch 47 | time = 136s | loss gen = 0.3578 | loss lab = 0.2290 | loss unl = 0.0225 | train acc = 0.9971| test acc = 0.7024 | test acc ema = 0.7269
# Epoch 48 | time = 136s | loss gen = 0.3544 | loss lab = 0.2271 | loss unl = 0.0217 | train acc = 0.9978| test acc = 0.7083 | test acc ema = 0.7295
# Epoch 49 | time = 136s | loss gen = 0.3533 | loss lab = 0.2244 | loss unl = 0.0218 | train acc = 0.9972| test acc = 0.7169 | test acc ema = 0.7308
# Epoch 50 | time = 136s | loss gen = 0.3486 | loss lab = 0.2250 | loss unl = 0.0208 | train acc = 0.9975| test acc = 0.7163 | test acc ema = 0.7319
# Model saved in file: ./log/cifar/model-50
# Epoch 51 | time = 137s | loss gen = 0.3444 | loss lab = 0.2244 | loss unl = 0.0204 | train acc = 0.9977| test acc = 0.7291 | test acc ema = 0.7347
# Epoch 52 | time = 137s | loss gen = 0.3441 | loss lab = 0.2203 | loss unl = 0.0209 | train acc = 0.9976| test acc = 0.7277 | test acc ema = 0.7383
# Epoch 53 | time = 136s | loss gen = 0.3429 | loss lab = 0.2192 | loss unl = 0.0195 | train acc = 0.9979| test acc = 0.7249 | test acc ema = 0.7401
# Epoch 54 | time = 136s | loss gen = 0.3397 | loss lab = 0.2191 | loss unl = 0.0198 | train acc = 0.9978| test acc = 0.7281 | test acc ema = 0.7405
# Epoch 55 | time = 136s | loss gen = 0.3387 | loss lab = 0.2186 | loss unl = 0.0190 | train acc = 0.9980| test acc = 0.7237 | test acc ema = 0.7403
# Epoch 56 | time = 136s | loss gen = 0.3347 | loss lab = 0.2161 | loss unl = 0.0187 | train acc = 0.9980| test acc = 0.7179 | test acc ema = 0.7427
# Epoch 57 | time = 137s | loss gen = 0.3307 | loss lab = 0.2177 | loss unl = 0.0183 | train acc = 0.9979| test acc = 0.7329 | test acc ema = 0.7430
# Epoch 58 | time = 136s | loss gen = 0.3306 | loss lab = 0.2163 | loss unl = 0.0186 | train acc = 0.9981| test acc = 0.7435 | test acc ema = 0.7451
# Epoch 59 | time = 136s | loss gen = 0.3277 | loss lab = 0.2144 | loss unl = 0.0186 | train acc = 0.9980| test acc = 0.7481 | test acc ema = 0.7492
# Epoch 60 | time = 136s | loss gen = 0.3241 | loss lab = 0.2124 | loss unl = 0.0176 | train acc = 0.9982| test acc = 0.7364 | test acc ema = 0.7509
# Epoch 61 | time = 136s | loss gen = 0.3229 | loss lab = 0.2117 | loss unl = 0.0176 | train acc = 0.9981| test acc = 0.7293 | test acc ema = 0.7522
# Epoch 62 | time = 136s | loss gen = 0.3192 | loss lab = 0.2093 | loss unl = 0.0168 | train acc = 0.9983| test acc = 0.7314 | test acc ema = 0.7523
# Epoch 63 | time = 136s | loss gen = 0.3170 | loss lab = 0.2093 | loss unl = 0.0169 | train acc = 0.9983| test acc = 0.7594 | test acc ema = 0.7538
# Epoch 64 | time = 136s | loss gen = 0.3171 | loss lab = 0.2083 | loss unl = 0.0170 | train acc = 0.9984| test acc = 0.7283 | test acc ema = 0.7556
# Epoch 65 | time = 136s | loss gen = 0.3157 | loss lab = 0.2036 | loss unl = 0.0160 | train acc = 0.9983| test acc = 0.7410 | test acc ema = 0.7567
# Epoch 66 | time = 136s | loss gen = 0.3132 | loss lab = 0.2041 | loss unl = 0.0159 | train acc = 0.9986| test acc = 0.7246 | test acc ema = 0.7570
# Epoch 67 | time = 136s | loss gen = 0.3144 | loss lab = 0.2039 | loss unl = 0.0156 | train acc = 0.9984| test acc = 0.7481 | test acc ema = 0.7570
# Epoch 68 | time = 136s | loss gen = 0.3123 | loss lab = 0.1979 | loss unl = 0.0154 | train acc = 0.9987| test acc = 0.7513 | test acc ema = 0.7587
# Epoch 69 | time = 136s | loss gen = 0.3107 | loss lab = 0.2009 | loss unl = 0.0152 | train acc = 0.9987| test acc = 0.7270 | test acc ema = 0.7618
# Epoch 70 | time = 136s | loss gen = 0.3077 | loss lab = 0.1973 | loss unl = 0.0161 | train acc = 0.9979| test acc = 0.7520 | test acc ema = 0.7638
# Epoch 71 | time = 136s | loss gen = 0.3062 | loss lab = 0.1975 | loss unl = 0.0147 | train acc = 0.9987| test acc = 0.7573 | test acc ema = 0.7648
# Epoch 72 | time = 136s | loss gen = 0.3069 | loss lab = 0.1961 | loss unl = 0.0142 | train acc = 0.9988| test acc = 0.7576 | test acc ema = 0.7645
# Epoch 73 | time = 136s | loss gen = 0.3042 | loss lab = 0.1928 | loss unl = 0.0145 | train acc = 0.9988| test acc = 0.7697 | test acc ema = 0.7651
# Epoch 74 | time = 136s | loss gen = 0.3023 | loss lab = 0.1943 | loss unl = 0.0147 | train acc = 0.9984| test acc = 0.7594 | test acc ema = 0.7669
# Epoch 75 | time = 136s | loss gen = 0.3029 | loss lab = 0.1919 | loss unl = 0.0144 | train acc = 0.9984| test acc = 0.7565 | test acc ema = 0.7676
# Epoch 76 | time = 136s | loss gen = 0.3009 | loss lab = 0.1911 | loss unl = 0.0141 | train acc = 0.9986| test acc = 0.7484 | test acc ema = 0.7687
# Epoch 77 | time = 136s | loss gen = 0.2990 | loss lab = 0.1909 | loss unl = 0.0143 | train acc = 0.9986| test acc = 0.7654 | test acc ema = 0.7690
# Epoch 78 | time = 136s | loss gen = 0.2956 | loss lab = 0.1900 | loss unl = 0.0138 | train acc = 0.9986| test acc = 0.7583 | test acc ema = 0.7703
# Epoch 79 | time = 136s | loss gen = 0.2950 | loss lab = 0.1888 | loss unl = 0.0137 | train acc = 0.9988| test acc = 0.7554 | test acc ema = 0.7716
# Epoch 80 | time = 136s | loss gen = 0.2956 | loss lab = 0.1864 | loss unl = 0.0140 | train acc = 0.9984| test acc = 0.7482 | test acc ema = 0.7711
# Epoch 81 | time = 136s | loss gen = 0.2924 | loss lab = 0.1882 | loss unl = 0.0136 | train acc = 0.9984| test acc = 0.7552 | test acc ema = 0.7716
# Epoch 82 | time = 136s | loss gen = 0.2910 | loss lab = 0.1881 | loss unl = 0.0135 | train acc = 0.9986| test acc = 0.7532 | test acc ema = 0.7719
# Epoch 83 | time = 136s | loss gen = 0.2906 | loss lab = 0.1870 | loss unl = 0.0130 | train acc = 0.9989| test acc = 0.7667 | test acc ema = 0.7727
# Epoch 84 | time = 136s | loss gen = 0.2902 | loss lab = 0.1830 | loss unl = 0.0125 | train acc = 0.9989| test acc = 0.7578 | test acc ema = 0.7725
# Epoch 85 | time = 136s | loss gen = 0.2872 | loss lab = 0.1832 | loss unl = 0.0131 | train acc = 0.9988| test acc = 0.7680 | test acc ema = 0.7735
# Epoch 86 | time = 136s | loss gen = 0.2875 | loss lab = 0.1814 | loss unl = 0.0129 | train acc = 0.9988| test acc = 0.7644 | test acc ema = 0.7757
# Epoch 87 | time = 136s | loss gen = 0.2869 | loss lab = 0.1774 | loss unl = 0.0124 | train acc = 0.9989| test acc = 0.7577 | test acc ema = 0.7771
# Epoch 88 | time = 136s | loss gen = 0.2856 | loss lab = 0.1797 | loss unl = 0.0124 | train acc = 0.9989| test acc = 0.7665 | test acc ema = 0.7777
# Epoch 89 | time = 136s | loss gen = 0.2849 | loss lab = 0.1775 | loss unl = 0.0125 | train acc = 0.9987| test acc = 0.7579 | test acc ema = 0.7781
# Epoch 90 | time = 136s | loss gen = 0.2850 | loss lab = 0.1765 | loss unl = 0.0119 | train acc = 0.9990| test acc = 0.7737 | test acc ema = 0.7800
# Epoch 91 | time = 136s | loss gen = 0.2828 | loss lab = 0.1762 | loss unl = 0.0124 | train acc = 0.9989| test acc = 0.7690 | test acc ema = 0.7794
# Epoch 92 | time = 136s | loss gen = 0.2819 | loss lab = 0.1776 | loss unl = 0.0121 | train acc = 0.9990| test acc = 0.7767 | test acc ema = 0.7831
# Epoch 93 | time = 136s | loss gen = 0.2805 | loss lab = 0.1738 | loss unl = 0.0118 | train acc = 0.9989| test acc = 0.7623 | test acc ema = 0.7845
# Epoch 94 | time = 136s | loss gen = 0.2782 | loss lab = 0.1720 | loss unl = 0.0119 | train acc = 0.9989| test acc = 0.7764 | test acc ema = 0.7859
# Epoch 95 | time = 136s | loss gen = 0.2769 | loss lab = 0.1725 | loss unl = 0.0116 | train acc = 0.9988| test acc = 0.7851 | test acc ema = 0.7860
# Epoch 96 | time = 136s | loss gen = 0.2788 | loss lab = 0.1693 | loss unl = 0.0113 | train acc = 0.9990| test acc = 0.7710 | test acc ema = 0.7861
# Epoch 97 | time = 136s | loss gen = 0.2763 | loss lab = 0.1673 | loss unl = 0.0107 | train acc = 0.9992| test acc = 0.7695 | test acc ema = 0.7860
# Epoch 98 | time = 136s | loss gen = 0.2755 | loss lab = 0.1687 | loss unl = 0.0116 | train acc = 0.9990| test acc = 0.7751 | test acc ema = 0.7863
# Epoch 99 | time = 136s | loss gen = 0.2762 | loss lab = 0.1707 | loss unl = 0.0113 | train acc = 0.9990| test acc = 0.7638 | test acc ema = 0.7870
# Epoch 100 | time = 136s | loss gen = 0.2751 | loss lab = 0.1674 | loss unl = 0.0108 | train acc = 0.9993| test acc = 0.7756 | test acc ema = 0.7886
# Model saved in file: ./log/cifar/model-100
# Epoch 101 | time = 136s | loss gen = 0.2756 | loss lab = 0.1668 | loss unl = 0.0111 | train acc = 0.9991| test acc = 0.7788 | test acc ema = 0.7886
# Epoch 102 | time = 136s | loss gen = 0.2737 | loss lab = 0.1653 | loss unl = 0.0107 | train acc = 0.9990| test acc = 0.7703 | test acc ema = 0.7901
# Epoch 103 | time = 136s | loss gen = 0.2732 | loss lab = 0.1653 | loss unl = 0.0107 | train acc = 0.9990| test acc = 0.7796 | test acc ema = 0.7908
# Epoch 104 | time = 136s | loss gen = 0.2722 | loss lab = 0.1666 | loss unl = 0.0106 | train acc = 0.9991| test acc = 0.7781 | test acc ema = 0.7914
# Epoch 105 | time = 136s | loss gen = 0.2722 | loss lab = 0.1639 | loss unl = 0.0104 | train acc = 0.9992| test acc = 0.7707 | test acc ema = 0.7919
# Epoch 106 | time = 136s | loss gen = 0.2711 | loss lab = 0.1619 | loss unl = 0.0099 | train acc = 0.9993| test acc = 0.7721 | test acc ema = 0.7913
# Epoch 107 | time = 136s | loss gen = 0.2697 | loss lab = 0.1646 | loss unl = 0.0102 | train acc = 0.9991| test acc = 0.7676 | test acc ema = 0.7913
# Epoch 108 | time = 136s | loss gen = 0.2693 | loss lab = 0.1612 | loss unl = 0.0102 | train acc = 0.9989| test acc = 0.7761 | test acc ema = 0.7932
# Epoch 109 | time = 136s | loss gen = 0.2685 | loss lab = 0.1634 | loss unl = 0.0105 | train acc = 0.9988| test acc = 0.7694 | test acc ema = 0.7937
# Epoch 110 | time = 136s | loss gen = 0.2684 | loss lab = 0.1610 | loss unl = 0.0111 | train acc = 0.9989| test acc = 0.7905 | test acc ema = 0.7956
# Epoch 111 | time = 136s | loss gen = 0.2678 | loss lab = 0.1606 | loss unl = 0.0100 | train acc = 0.9991| test acc = 0.7866 | test acc ema = 0.7953
# Epoch 112 | time = 136s | loss gen = 0.2655 | loss lab = 0.1590 | loss unl = 0.0103 | train acc = 0.9991| test acc = 0.7769 | test acc ema = 0.7961
# Epoch 113 | time = 136s | loss gen = 0.2645 | loss lab = 0.1596 | loss unl = 0.0103 | train acc = 0.9990| test acc = 0.7789 | test acc ema = 0.7968
# Epoch 114 | time = 136s | loss gen = 0.2639 | loss lab = 0.1601 | loss unl = 0.0100 | train acc = 0.9990| test acc = 0.7877 | test acc ema = 0.7984
# Epoch 115 | time = 136s | loss gen = 0.2636 | loss lab = 0.1577 | loss unl = 0.0096 | train acc = 0.9991| test acc = 0.7882 | test acc ema = 0.7996
# Epoch 116 | time = 136s | loss gen = 0.2653 | loss lab = 0.1538 | loss unl = 0.0096 | train acc = 0.9991| test acc = 0.7760 | test acc ema = 0.7991
# Epoch 117 | time = 136s | loss gen = 0.2626 | loss lab = 0.1570 | loss unl = 0.0096 | train acc = 0.9993| test acc = 0.7906 | test acc ema = 0.7986
# Epoch 118 | time = 136s | loss gen = 0.2620 | loss lab = 0.1563 | loss unl = 0.0096 | train acc = 0.9993| test acc = 0.7910 | test acc ema = 0.7993
# Epoch 119 | time = 136s | loss gen = 0.2592 | loss lab = 0.1561 | loss unl = 0.0096 | train acc = 0.9991| test acc = 0.7895 | test acc ema = 0.8013
# Epoch 120 | time = 136s | loss gen = 0.2613 | loss lab = 0.1528 | loss unl = 0.0093 | train acc = 0.9992| test acc = 0.7896 | test acc ema = 0.7983
# Epoch 121 | time = 136s | loss gen = 0.2582 | loss lab = 0.1574 | loss unl = 0.0094 | train acc = 0.9992| test acc = 0.7831 | test acc ema = 0.7988
# Epoch 122 | time = 136s | loss gen = 0.2610 | loss lab = 0.1525 | loss unl = 0.0091 | train acc = 0.9992| test acc = 0.7923 | test acc ema = 0.7994
# Epoch 123 | time = 136s | loss gen = 0.2573 | loss lab = 0.1535 | loss unl = 0.0094 | train acc = 0.9991| test acc = 0.7938 | test acc ema = 0.8020
# Epoch 124 | time = 136s | loss gen = 0.2575 | loss lab = 0.1521 | loss unl = 0.0091 | train acc = 0.9991| test acc = 0.7862 | test acc ema = 0.8030
# Epoch 125 | time = 136s | loss gen = 0.2563 | loss lab = 0.1521 | loss unl = 0.0086 | train acc = 0.9994| test acc = 0.7919 | test acc ema = 0.8030
# Epoch 126 | time = 136s | loss gen = 0.2559 | loss lab = 0.1547 | loss unl = 0.0094 | train acc = 0.9990| test acc = 0.7936 | test acc ema = 0.8032
# Epoch 127 | time = 136s | loss gen = 0.2552 | loss lab = 0.1529 | loss unl = 0.0094 | train acc = 0.9991| test acc = 0.7806 | test acc ema = 0.8032
# Epoch 128 | time = 136s | loss gen = 0.2543 | loss lab = 0.1501 | loss unl = 0.0091 | train acc = 0.9991| test acc = 0.7821 | test acc ema = 0.8047
# Epoch 129 | time = 136s | loss gen = 0.2552 | loss lab = 0.1510 | loss unl = 0.0092 | train acc = 0.9992| test acc = 0.7944 | test acc ema = 0.8060
# Epoch 130 | time = 136s | loss gen = 0.2555 | loss lab = 0.1503 | loss unl = 0.0091 | train acc = 0.9992| test acc = 0.7945 | test acc ema = 0.8048
# Epoch 131 | time = 136s | loss gen = 0.2522 | loss lab = 0.1499 | loss unl = 0.0085 | train acc = 0.9994| test acc = 0.7989 | test acc ema = 0.8052
# Epoch 132 | time = 136s | loss gen = 0.2528 | loss lab = 0.1501 | loss unl = 0.0087 | train acc = 0.9992| test acc = 0.7906 | test acc ema = 0.8059
# Epoch 133 | time = 136s | loss gen = 0.2531 | loss lab = 0.1471 | loss unl = 0.0085 | train acc = 0.9994| test acc = 0.7939 | test acc ema = 0.8054
# Epoch 134 | time = 136s | loss gen = 0.2525 | loss lab = 0.1503 | loss unl = 0.0089 | train acc = 0.9992| test acc = 0.7970 | test acc ema = 0.8056
# Epoch 135 | time = 136s | loss gen = 0.2527 | loss lab = 0.1488 | loss unl = 0.0083 | train acc = 0.9994| test acc = 0.7927 | test acc ema = 0.8055
# Epoch 136 | time = 136s | loss gen = 0.2506 | loss lab = 0.1467 | loss unl = 0.0085 | train acc = 0.9993| test acc = 0.7922 | test acc ema = 0.8057
# Epoch 137 | time = 136s | loss gen = 0.2529 | loss lab = 0.1421 | loss unl = 0.0084 | train acc = 0.9992| test acc = 0.7866 | test acc ema = 0.8070
# Epoch 138 | time = 135s | loss gen = 0.2519 | loss lab = 0.1451 | loss unl = 0.0082 | train acc = 0.9993| test acc = 0.7882 | test acc ema = 0.8072
# Epoch 139 | time = 136s | loss gen = 0.2504 | loss lab = 0.1448 | loss unl = 0.0088 | train acc = 0.9992| test acc = 0.7974 | test acc ema = 0.8076
# Epoch 140 | time = 136s | loss gen = 0.2499 | loss lab = 0.1439 | loss unl = 0.0085 | train acc = 0.9992| test acc = 0.8037 | test acc ema = 0.8091
# Epoch 141 | time = 136s | loss gen = 0.2498 | loss lab = 0.1424 | loss unl = 0.0080 | train acc = 0.9994| test acc = 0.7980 | test acc ema = 0.8084
# Epoch 142 | time = 136s | loss gen = 0.2483 | loss lab = 0.1434 | loss unl = 0.0086 | train acc = 0.9991| test acc = 0.8042 | test acc ema = 0.8096
# Epoch 143 | time = 136s | loss gen = 0.2493 | loss lab = 0.1412 | loss unl = 0.0080 | train acc = 0.9993| test acc = 0.7940 | test acc ema = 0.8090
# Epoch 144 | time = 135s | loss gen = 0.2492 | loss lab = 0.1419 | loss unl = 0.0079 | train acc = 0.9994| test acc = 0.7970 | test acc ema = 0.8104
# Epoch 145 | time = 135s | loss gen = 0.2468 | loss lab = 0.1429 | loss unl = 0.0082 | train acc = 0.9993| test acc = 0.7930 | test acc ema = 0.8103
# Epoch 146 | time = 135s | loss gen = 0.2476 | loss lab = 0.1428 | loss unl = 0.0080 | train acc = 0.9994| test acc = 0.7968 | test acc ema = 0.8103
# Epoch 147 | time = 135s | loss gen = 0.2463 | loss lab = 0.1438 | loss unl = 0.0078 | train acc = 0.9993| test acc = 0.7969 | test acc ema = 0.8106
# Epoch 148 | time = 136s | loss gen = 0.2474 | loss lab = 0.1384 | loss unl = 0.0075 | train acc = 0.9995| test acc = 0.7954 | test acc ema = 0.8100
# Epoch 149 | time = 136s | loss gen = 0.2456 | loss lab = 0.1416 | loss unl = 0.0081 | train acc = 0.9993| test acc = 0.7964 | test acc ema = 0.8120
# Epoch 150 | time = 136s | loss gen = 0.2474 | loss lab = 0.1387 | loss unl = 0.0082 | train acc = 0.9991| test acc = 0.7982 | test acc ema = 0.8112
# Model saved in file: ./log/cifar/model-150
# Epoch 151 | time = 135s | loss gen = 0.2450 | loss lab = 0.1387 | loss unl = 0.0078 | train acc = 0.9993| test acc = 0.8100 | test acc ema = 0.8135
# Epoch 152 | time = 135s | loss gen = 0.2456 | loss lab = 0.1369 | loss unl = 0.0073 | train acc = 0.9995| test acc = 0.8089 | test acc ema = 0.8149
# Epoch 153 | time = 136s | loss gen = 0.2463 | loss lab = 0.1380 | loss unl = 0.0077 | train acc = 0.9994| test acc = 0.8040 | test acc ema = 0.8150
# Epoch 154 | time = 136s | loss gen = 0.2456 | loss lab = 0.1377 | loss unl = 0.0076 | train acc = 0.9995| test acc = 0.8026 | test acc ema = 0.8152
# Epoch 155 | time = 135s | loss gen = 0.2451 | loss lab = 0.1355 | loss unl = 0.0077 | train acc = 0.9993| test acc = 0.7961 | test acc ema = 0.8142
# Epoch 156 | time = 135s | loss gen = 0.2448 | loss lab = 0.1358 | loss unl = 0.0075 | train acc = 0.9994| test acc = 0.8014 | test acc ema = 0.8135
# Epoch 157 | time = 135s | loss gen = 0.2432 | loss lab = 0.1353 | loss unl = 0.0074 | train acc = 0.9996| test acc = 0.8028 | test acc ema = 0.8130
# Epoch 158 | time = 135s | loss gen = 0.2450 | loss lab = 0.1368 | loss unl = 0.0074 | train acc = 0.9993| test acc = 0.7988 | test acc ema = 0.8145
# Epoch 159 | time = 135s | loss gen = 0.2446 | loss lab = 0.1332 | loss unl = 0.0071 | train acc = 0.9995| test acc = 0.8085 | test acc ema = 0.8150
# Epoch 160 | time = 135s | loss gen = 0.2433 | loss lab = 0.1334 | loss unl = 0.0071 | train acc = 0.9996| test acc = 0.8011 | test acc ema = 0.8155
# Epoch 161 | time = 135s | loss gen = 0.2441 | loss lab = 0.1328 | loss unl = 0.0073 | train acc = 0.9994| test acc = 0.8037 | test acc ema = 0.8149
# Epoch 162 | time = 135s | loss gen = 0.2430 | loss lab = 0.1330 | loss unl = 0.0076 | train acc = 0.9994| test acc = 0.7969 | test acc ema = 0.8133
# Epoch 163 | time = 135s | loss gen = 0.2422 | loss lab = 0.1319 | loss unl = 0.0073 | train acc = 0.9994| test acc = 0.8005 | test acc ema = 0.8136
# Epoch 164 | time = 135s | loss gen = 0.2428 | loss lab = 0.1329 | loss unl = 0.0071 | train acc = 0.9993| test acc = 0.8019 | test acc ema = 0.8135
# Epoch 165 | time = 135s | loss gen = 0.2409 | loss lab = 0.1349 | loss unl = 0.0072 | train acc = 0.9994| test acc = 0.7994 | test acc ema = 0.8143
# Epoch 166 | time = 135s | loss gen = 0.2408 | loss lab = 0.1309 | loss unl = 0.0069 | train acc = 0.9994| test acc = 0.8029 | test acc ema = 0.8147
# Epoch 167 | time = 135s | loss gen = 0.2400 | loss lab = 0.1330 | loss unl = 0.0070 | train acc = 0.9994| test acc = 0.8033 | test acc ema = 0.8155
# Epoch 168 | time = 135s | loss gen = 0.2402 | loss lab = 0.1319 | loss unl = 0.0070 | train acc = 0.9996| test acc = 0.8095 | test acc ema = 0.8151
# Epoch 169 | time = 135s | loss gen = 0.2396 | loss lab = 0.1315 | loss unl = 0.0071 | train acc = 0.9994| test acc = 0.8063 | test acc ema = 0.8170
# Epoch 170 | time = 136s | loss gen = 0.2387 | loss lab = 0.1330 | loss unl = 0.0066 | train acc = 0.9995| test acc = 0.8066 | test acc ema = 0.8177
# Epoch 171 | time = 135s | loss gen = 0.2390 | loss lab = 0.1344 | loss unl = 0.0072 | train acc = 0.9993| test acc = 0.7988 | test acc ema = 0.8182
# Epoch 172 | time = 135s | loss gen = 0.2376 | loss lab = 0.1297 | loss unl = 0.0068 | train acc = 0.9994| test acc = 0.8052 | test acc ema = 0.8183
# Epoch 173 | time = 136s | loss gen = 0.2373 | loss lab = 0.1299 | loss unl = 0.0070 | train acc = 0.9995| test acc = 0.7974 | test acc ema = 0.8189
# Epoch 174 | time = 136s | loss gen = 0.2381 | loss lab = 0.1287 | loss unl = 0.0066 | train acc = 0.9996| test acc = 0.8102 | test acc ema = 0.8187
# Epoch 175 | time = 136s | loss gen = 0.2374 | loss lab = 0.1297 | loss unl = 0.0069 | train acc = 0.9996| test acc = 0.7981 | test acc ema = 0.8186
# Epoch 176 | time = 136s | loss gen = 0.2364 | loss lab = 0.1289 | loss unl = 0.0071 | train acc = 0.9991| test acc = 0.8102 | test acc ema = 0.8194
# Epoch 177 | time = 136s | loss gen = 0.2374 | loss lab = 0.1321 | loss unl = 0.0069 | train acc = 0.9994| test acc = 0.8019 | test acc ema = 0.8204
# Epoch 178 | time = 135s | loss gen = 0.2353 | loss lab = 0.1303 | loss unl = 0.0071 | train acc = 0.9994| test acc = 0.8095 | test acc ema = 0.8215
# Epoch 179 | time = 135s | loss gen = 0.2355 | loss lab = 0.1286 | loss unl = 0.0066 | train acc = 0.9996| test acc = 0.8008 | test acc ema = 0.8195
# Epoch 180 | time = 135s | loss gen = 0.2376 | loss lab = 0.1278 | loss unl = 0.0067 | train acc = 0.9995| test acc = 0.8059 | test acc ema = 0.8201
# Epoch 181 | time = 135s | loss gen = 0.2367 | loss lab = 0.1271 | loss unl = 0.0068 | train acc = 0.9994| test acc = 0.8067 | test acc ema = 0.8215
# Epoch 182 | time = 135s | loss gen = 0.2364 | loss lab = 0.1254 | loss unl = 0.0063 | train acc = 0.9996| test acc = 0.7995 | test acc ema = 0.8218
# Epoch 183 | time = 135s | loss gen = 0.2369 | loss lab = 0.1250 | loss unl = 0.0068 | train acc = 0.9995| test acc = 0.8099 | test acc ema = 0.8216
# Epoch 184 | time = 136s | loss gen = 0.2364 | loss lab = 0.1261 | loss unl = 0.0068 | train acc = 0.9994| test acc = 0.8173 | test acc ema = 0.8222
# Epoch 185 | time = 135s | loss gen = 0.2336 | loss lab = 0.1256 | loss unl = 0.0066 | train acc = 0.9995| test acc = 0.8067 | test acc ema = 0.8228
# Epoch 186 | time = 135s | loss gen = 0.2335 | loss lab = 0.1247 | loss unl = 0.0065 | train acc = 0.9994| test acc = 0.8123 | test acc ema = 0.8222
# Epoch 187 | time = 135s | loss gen = 0.2361 | loss lab = 0.1250 | loss unl = 0.0064 | train acc = 0.9996| test acc = 0.8011 | test acc ema = 0.8217
# Epoch 188 | time = 135s | loss gen = 0.2340 | loss lab = 0.1245 | loss unl = 0.0065 | train acc = 0.9994| test acc = 0.8173 | test acc ema = 0.8225
# Epoch 189 | time = 135s | loss gen = 0.2335 | loss lab = 0.1249 | loss unl = 0.0064 | train acc = 0.9996| test acc = 0.8183 | test acc ema = 0.8231
# Epoch 190 | time = 136s | loss gen = 0.2345 | loss lab = 0.1226 | loss unl = 0.0061 | train acc = 0.9996| test acc = 0.8125 | test acc ema = 0.8224
# Epoch 191 | time = 135s | loss gen = 0.2324 | loss lab = 0.1235 | loss unl = 0.0064 | train acc = 0.9996| test acc = 0.8123 | test acc ema = 0.8217
# Epoch 192 | time = 135s | loss gen = 0.2324 | loss lab = 0.1228 | loss unl = 0.0064 | train acc = 0.9993| test acc = 0.8054 | test acc ema = 0.8222
# Epoch 193 | time = 135s | loss gen = 0.2346 | loss lab = 0.1238 | loss unl = 0.0062 | train acc = 0.9995| test acc = 0.8118 | test acc ema = 0.8221
# Epoch 194 | time = 135s | loss gen = 0.2335 | loss lab = 0.1211 | loss unl = 0.0062 | train acc = 0.9996| test acc = 0.8099 | test acc ema = 0.8227
# Epoch 195 | time = 135s | loss gen = 0.2341 | loss lab = 0.1208 | loss unl = 0.0065 | train acc = 0.9995| test acc = 0.8093 | test acc ema = 0.8230
# Epoch 196 | time = 136s | loss gen = 0.2306 | loss lab = 0.1231 | loss unl = 0.0065 | train acc = 0.9995| test acc = 0.8134 | test acc ema = 0.8233
# Epoch 197 | time = 135s | loss gen = 0.2314 | loss lab = 0.1192 | loss unl = 0.0061 | train acc = 0.9995| test acc = 0.8085 | test acc ema = 0.8227
# Epoch 198 | time = 136s | loss gen = 0.2313 | loss lab = 0.1213 | loss unl = 0.0062 | train acc = 0.9995| test acc = 0.7994 | test acc ema = 0.8215
# Epoch 199 | time = 135s | loss gen = 0.2322 | loss lab = 0.1197 | loss unl = 0.0056 | train acc = 0.9996| test acc = 0.8059 | test acc ema = 0.8221
# Epoch 200 | time = 135s | loss gen = 0.2313 | loss lab = 0.1186 | loss unl = 0.0059 | train acc = 0.9995| test acc = 0.8075 | test acc ema = 0.8227
# Model saved in file: ./log/cifar/model-200
# Epoch 201 | time = 135s | loss gen = 0.2296 | loss lab = 0.1203 | loss unl = 0.0064 | train acc = 0.9994| test acc = 0.8075 | test acc ema = 0.8229
# Epoch 202 | time = 135s | loss gen = 0.2301 | loss lab = 0.1247 | loss unl = 0.0062 | train acc = 0.9997| test acc = 0.8056 | test acc ema = 0.8230
# Epoch 203 | time = 135s | loss gen = 0.2308 | loss lab = 0.1213 | loss unl = 0.0060 | train acc = 0.9996| test acc = 0.8139 | test acc ema = 0.8238
# Epoch 204 | time = 135s | loss gen = 0.2309 | loss lab = 0.1208 | loss unl = 0.0059 | train acc = 0.9997| test acc = 0.8061 | test acc ema = 0.8235
# Epoch 205 | time = 135s | loss gen = 0.2300 | loss lab = 0.1194 | loss unl = 0.0060 | train acc = 0.9996| test acc = 0.8079 | test acc ema = 0.8241
# Epoch 206 | time = 136s | loss gen = 0.2269 | loss lab = 0.1201 | loss unl = 0.0060 | train acc = 0.9995| test acc = 0.8018 | test acc ema = 0.8255
# Epoch 207 | time = 136s | loss gen = 0.2286 | loss lab = 0.1210 | loss unl = 0.0060 | train acc = 0.9997| test acc = 0.8174 | test acc ema = 0.8248
# Epoch 208 | time = 136s | loss gen = 0.2297 | loss lab = 0.1187 | loss unl = 0.0059 | train acc = 0.9996| test acc = 0.8134 | test acc ema = 0.8255
# Epoch 209 | time = 135s | loss gen = 0.2305 | loss lab = 0.1174 | loss unl = 0.0055 | train acc = 0.9997| test acc = 0.8138 | test acc ema = 0.8262
# Epoch 210 | time = 136s | loss gen = 0.2275 | loss lab = 0.1175 | loss unl = 0.0061 | train acc = 0.9995| test acc = 0.8173 | test acc ema = 0.8263
# Epoch 211 | time = 136s | loss gen = 0.2301 | loss lab = 0.1157 | loss unl = 0.0060 | train acc = 0.9996| test acc = 0.8115 | test acc ema = 0.8265
# Epoch 212 | time = 136s | loss gen = 0.2297 | loss lab = 0.1166 | loss unl = 0.0056 | train acc = 0.9996| test acc = 0.8185 | test acc ema = 0.8270
# Epoch 213 | time = 136s | loss gen = 0.2270 | loss lab = 0.1189 | loss unl = 0.0057 | train acc = 0.9995| test acc = 0.8170 | test acc ema = 0.8265
# Epoch 214 | time = 136s | loss gen = 0.2272 | loss lab = 0.1175 | loss unl = 0.0058 | train acc = 0.9995| test acc = 0.8169 | test acc ema = 0.8257
# Epoch 215 | time = 135s | loss gen = 0.2287 | loss lab = 0.1159 | loss unl = 0.0060 | train acc = 0.9994| test acc = 0.8159 | test acc ema = 0.8261
# Epoch 216 | time = 136s | loss gen = 0.2265 | loss lab = 0.1162 | loss unl = 0.0054 | train acc = 0.9997| test acc = 0.8166 | test acc ema = 0.8254
# Epoch 217 | time = 136s | loss gen = 0.2258 | loss lab = 0.1174 | loss unl = 0.0063 | train acc = 0.9994| test acc = 0.8097 | test acc ema = 0.8254
# Epoch 218 | time = 135s | loss gen = 0.2265 | loss lab = 0.1154 | loss unl = 0.0054 | train acc = 0.9997| test acc = 0.8189 | test acc ema = 0.8258
# Epoch 219 | time = 135s | loss gen = 0.2276 | loss lab = 0.1137 | loss unl = 0.0060 | train acc = 0.9993| test acc = 0.8149 | test acc ema = 0.8251
# Epoch 220 | time = 136s | loss gen = 0.2276 | loss lab = 0.1167 | loss unl = 0.0058 | train acc = 0.9995| test acc = 0.8176 | test acc ema = 0.8264
# Epoch 221 | time = 135s | loss gen = 0.2284 | loss lab = 0.1163 | loss unl = 0.0060 | train acc = 0.9994| test acc = 0.8163 | test acc ema = 0.8257
# Epoch 222 | time = 136s | loss gen = 0.2259 | loss lab = 0.1149 | loss unl = 0.0057 | train acc = 0.9996| test acc = 0.8212 | test acc ema = 0.8262
# Epoch 223 | time = 136s | loss gen = 0.2260 | loss lab = 0.1145 | loss unl = 0.0058 | train acc = 0.9995| test acc = 0.8120 | test acc ema = 0.8273
# Epoch 224 | time = 136s | loss gen = 0.2262 | loss lab = 0.1138 | loss unl = 0.0057 | train acc = 0.9995| test acc = 0.8180 | test acc ema = 0.8266
# Epoch 225 | time = 136s | loss gen = 0.2248 | loss lab = 0.1134 | loss unl = 0.0054 | train acc = 0.9995| test acc = 0.8143 | test acc ema = 0.8264
# Epoch 226 | time = 136s | loss gen = 0.2255 | loss lab = 0.1122 | loss unl = 0.0052 | train acc = 0.9997| test acc = 0.8166 | test acc ema = 0.8251
# Epoch 227 | time = 136s | loss gen = 0.2256 | loss lab = 0.1137 | loss unl = 0.0059 | train acc = 0.9994| test acc = 0.8106 | test acc ema = 0.8258
# Epoch 228 | time = 136s | loss gen = 0.2256 | loss lab = 0.1151 | loss unl = 0.0053 | train acc = 0.9997| test acc = 0.8186 | test acc ema = 0.8246
# Epoch 229 | time = 136s | loss gen = 0.2257 | loss lab = 0.1164 | loss unl = 0.0056 | train acc = 0.9995| test acc = 0.8239 | test acc ema = 0.8266
# Epoch 230 | time = 136s | loss gen = 0.2254 | loss lab = 0.1128 | loss unl = 0.0054 | train acc = 0.9996| test acc = 0.8093 | test acc ema = 0.8257
# Epoch 231 | time = 136s | loss gen = 0.2250 | loss lab = 0.1143 | loss unl = 0.0055 | train acc = 0.9997| test acc = 0.8165 | test acc ema = 0.8262
# Epoch 232 | time = 136s | loss gen = 0.2233 | loss lab = 0.1122 | loss unl = 0.0055 | train acc = 0.9996| test acc = 0.8193 | test acc ema = 0.8269
# Epoch 233 | time = 136s | loss gen = 0.2260 | loss lab = 0.1118 | loss unl = 0.0057 | train acc = 0.9996| test acc = 0.8085 | test acc ema = 0.8273
# Epoch 234 | time = 136s | loss gen = 0.2252 | loss lab = 0.1111 | loss unl = 0.0052 | train acc = 0.9997| test acc = 0.8200 | test acc ema = 0.8271
# Epoch 235 | time = 136s | loss gen = 0.2248 | loss lab = 0.1135 | loss unl = 0.0053 | train acc = 0.9996| test acc = 0.8147 | test acc ema = 0.8263
# Epoch 236 | time = 136s | loss gen = 0.2253 | loss lab = 0.1126 | loss unl = 0.0054 | train acc = 0.9996| test acc = 0.8180 | test acc ema = 0.8277
# Epoch 237 | time = 136s | loss gen = 0.2226 | loss lab = 0.1118 | loss unl = 0.0051 | train acc = 0.9996| test acc = 0.8226 | test acc ema = 0.8275
# Epoch 238 | time = 136s | loss gen = 0.2242 | loss lab = 0.1109 | loss unl = 0.0053 | train acc = 0.9995| test acc = 0.8149 | test acc ema = 0.8271
# Epoch 239 | time = 136s | loss gen = 0.2230 | loss lab = 0.1100 | loss unl = 0.0051 | train acc = 0.9995| test acc = 0.8200 | test acc ema = 0.8284
# Epoch 240 | time = 136s | loss gen = 0.2223 | loss lab = 0.1118 | loss unl = 0.0055 | train acc = 0.9995| test acc = 0.8194 | test acc ema = 0.8289
# Epoch 241 | time = 136s | loss gen = 0.2229 | loss lab = 0.1086 | loss unl = 0.0055 | train acc = 0.9996| test acc = 0.8109 | test acc ema = 0.8280
# Epoch 242 | time = 136s | loss gen = 0.2243 | loss lab = 0.1114 | loss unl = 0.0056 | train acc = 0.9996| test acc = 0.8224 | test acc ema = 0.8264
# Epoch 243 | time = 136s | loss gen = 0.2247 | loss lab = 0.1100 | loss unl = 0.0054 | train acc = 0.9995| test acc = 0.8269 | test acc ema = 0.8273
# Epoch 244 | time = 136s | loss gen = 0.2240 | loss lab = 0.1125 | loss unl = 0.0050 | train acc = 0.9998| test acc = 0.8195 | test acc ema = 0.8274
# Epoch 245 | time = 136s | loss gen = 0.2248 | loss lab = 0.1098 | loss unl = 0.0051 | train acc = 0.9997| test acc = 0.8220 | test acc ema = 0.8278
# Epoch 246 | time = 136s | loss gen = 0.2245 | loss lab = 0.1113 | loss unl = 0.0051 | train acc = 0.9998| test acc = 0.8212 | test acc ema = 0.8293
# Epoch 247 | time = 136s | loss gen = 0.2223 | loss lab = 0.1111 | loss unl = 0.0052 | train acc = 0.9996| test acc = 0.8228 | test acc ema = 0.8298
# Epoch 248 | time = 136s | loss gen = 0.2237 | loss lab = 0.1086 | loss unl = 0.0051 | train acc = 0.9996| test acc = 0.8171 | test acc ema = 0.8302
# Epoch 249 | time = 136s | loss gen = 0.2230 | loss lab = 0.1072 | loss unl = 0.0054 | train acc = 0.9995| test acc = 0.8274 | test acc ema = 0.8297
# Epoch 250 | time = 136s | loss gen = 0.2217 | loss lab = 0.1096 | loss unl = 0.0053 | train acc = 0.9996| test acc = 0.8203 | test acc ema = 0.8308
# Model saved in file: ./log/cifar/model-250
# Epoch 251 | time = 136s | loss gen = 0.2220 | loss lab = 0.1084 | loss unl = 0.0051 | train acc = 0.9996| test acc = 0.8117 | test acc ema = 0.8301
# Epoch 252 | time = 136s | loss gen = 0.2200 | loss lab = 0.1081 | loss unl = 0.0049 | train acc = 0.9997| test acc = 0.8122 | test acc ema = 0.8286
# Epoch 253 | time = 136s | loss gen = 0.2207 | loss lab = 0.1076 | loss unl = 0.0052 | train acc = 0.9997| test acc = 0.8204 | test acc ema = 0.8289
# Epoch 254 | time = 136s | loss gen = 0.2218 | loss lab = 0.1068 | loss unl = 0.0050 | train acc = 0.9996| test acc = 0.8165 | test acc ema = 0.8300
# Epoch 255 | time = 136s | loss gen = 0.2211 | loss lab = 0.1076 | loss unl = 0.0053 | train acc = 0.9995| test acc = 0.8190 | test acc ema = 0.8294
# Epoch 256 | time = 136s | loss gen = 0.2220 | loss lab = 0.1074 | loss unl = 0.0053 | train acc = 0.9994| test acc = 0.8194 | test acc ema = 0.8289
# Epoch 257 | time = 136s | loss gen = 0.2221 | loss lab = 0.1075 | loss unl = 0.0052 | train acc = 0.9995| test acc = 0.8224 | test acc ema = 0.8298
# Epoch 258 | time = 136s | loss gen = 0.2222 | loss lab = 0.1079 | loss unl = 0.0049 | train acc = 0.9997| test acc = 0.8195 | test acc ema = 0.8297
# Epoch 259 | time = 136s | loss gen = 0.2217 | loss lab = 0.1075 | loss unl = 0.0053 | train acc = 0.9995| test acc = 0.8172 | test acc ema = 0.8308
# Epoch 260 | time = 136s | loss gen = 0.2205 | loss lab = 0.1108 | loss unl = 0.0051 | train acc = 0.9996| test acc = 0.8281 | test acc ema = 0.8307
# Epoch 261 | time = 136s | loss gen = 0.2206 | loss lab = 0.1093 | loss unl = 0.0051 | train acc = 0.9995| test acc = 0.8209 | test acc ema = 0.8305
# Epoch 262 | time = 136s | loss gen = 0.2209 | loss lab = 0.1071 | loss unl = 0.0050 | train acc = 0.9996| test acc = 0.8169 | test acc ema = 0.8306
# Epoch 263 | time = 136s | loss gen = 0.2204 | loss lab = 0.1073 | loss unl = 0.0051 | train acc = 0.9997| test acc = 0.8249 | test acc ema = 0.8315
# Epoch 264 | time = 136s | loss gen = 0.2202 | loss lab = 0.1049 | loss unl = 0.0051 | train acc = 0.9996| test acc = 0.8149 | test acc ema = 0.8305
# Epoch 265 | time = 136s | loss gen = 0.2209 | loss lab = 0.1077 | loss unl = 0.0048 | train acc = 0.9997| test acc = 0.8142 | test acc ema = 0.8303
# Epoch 266 | time = 136s | loss gen = 0.2207 | loss lab = 0.1062 | loss unl = 0.0050 | train acc = 0.9996| test acc = 0.8203 | test acc ema = 0.8310
# Epoch 267 | time = 136s | loss gen = 0.2202 | loss lab = 0.1069 | loss unl = 0.0051 | train acc = 0.9995| test acc = 0.8228 | test acc ema = 0.8319
# Epoch 268 | time = 136s | loss gen = 0.2193 | loss lab = 0.1061 | loss unl = 0.0048 | train acc = 0.9998| test acc = 0.8116 | test acc ema = 0.8305
# Epoch 269 | time = 136s | loss gen = 0.2215 | loss lab = 0.1049 | loss unl = 0.0051 | train acc = 0.9996| test acc = 0.8094 | test acc ema = 0.8310
# Epoch 270 | time = 136s | loss gen = 0.2231 | loss lab = 0.1056 | loss unl = 0.0049 | train acc = 0.9994| test acc = 0.8213 | test acc ema = 0.8308
# Epoch 271 | time = 136s | loss gen = 0.2204 | loss lab = 0.1078 | loss unl = 0.0052 | train acc = 0.9996| test acc = 0.8160 | test acc ema = 0.8320
# Epoch 272 | time = 136s | loss gen = 0.2213 | loss lab = 0.1040 | loss unl = 0.0047 | train acc = 0.9997| test acc = 0.8266 | test acc ema = 0.8316
# Epoch 273 | time = 136s | loss gen = 0.2188 | loss lab = 0.1059 | loss unl = 0.0049 | train acc = 0.9996| test acc = 0.8277 | test acc ema = 0.8319
# Epoch 274 | time = 136s | loss gen = 0.2189 | loss lab = 0.1060 | loss unl = 0.0049 | train acc = 0.9996| test acc = 0.8214 | test acc ema = 0.8335
# Epoch 275 | time = 136s | loss gen = 0.2201 | loss lab = 0.1045 | loss unl = 0.0050 | train acc = 0.9995| test acc = 0.8289 | test acc ema = 0.8334
# Epoch 276 | time = 136s | loss gen = 0.2207 | loss lab = 0.1035 | loss unl = 0.0049 | train acc = 0.9995| test acc = 0.8287 | test acc ema = 0.8327
# Epoch 277 | time = 136s | loss gen = 0.2188 | loss lab = 0.1027 | loss unl = 0.0047 | train acc = 0.9996| test acc = 0.8202 | test acc ema = 0.8328
# Epoch 278 | time = 136s | loss gen = 0.2205 | loss lab = 0.1038 | loss unl = 0.0052 | train acc = 0.9995| test acc = 0.8171 | test acc ema = 0.8334
# Epoch 279 | time = 136s | loss gen = 0.2190 | loss lab = 0.1031 | loss unl = 0.0046 | train acc = 0.9997| test acc = 0.8168 | test acc ema = 0.8323
# Epoch 280 | time = 136s | loss gen = 0.2205 | loss lab = 0.1043 | loss unl = 0.0050 | train acc = 0.9994| test acc = 0.8243 | test acc ema = 0.8325
# Epoch 281 | time = 136s | loss gen = 0.2193 | loss lab = 0.1035 | loss unl = 0.0047 | train acc = 0.9997| test acc = 0.8271 | test acc ema = 0.8327
# Epoch 282 | time = 136s | loss gen = 0.2195 | loss lab = 0.1043 | loss unl = 0.0045 | train acc = 0.9997| test acc = 0.8246 | test acc ema = 0.8327
# Epoch 283 | time = 136s | loss gen = 0.2198 | loss lab = 0.1021 | loss unl = 0.0047 | train acc = 0.9996| test acc = 0.8287 | test acc ema = 0.8321
# Epoch 284 | time = 136s | loss gen = 0.2190 | loss lab = 0.1028 | loss unl = 0.0048 | train acc = 0.9995| test acc = 0.8238 | test acc ema = 0.8333
# Epoch 285 | time = 136s | loss gen = 0.2174 | loss lab = 0.1022 | loss unl = 0.0047 | train acc = 0.9997| test acc = 0.8261 | test acc ema = 0.8321
# Epoch 286 | time = 136s | loss gen = 0.2179 | loss lab = 0.1018 | loss unl = 0.0045 | train acc = 0.9997| test acc = 0.8287 | test acc ema = 0.8328
# Epoch 287 | time = 136s | loss gen = 0.2188 | loss lab = 0.1013 | loss unl = 0.0048 | train acc = 0.9997| test acc = 0.8250 | test acc ema = 0.8323
# Epoch 288 | time = 136s | loss gen = 0.2184 | loss lab = 0.1022 | loss unl = 0.0047 | train acc = 0.9997| test acc = 0.8266 | test acc ema = 0.8331
# Epoch 289 | time = 136s | loss gen = 0.2176 | loss lab = 0.1018 | loss unl = 0.0046 | train acc = 0.9996| test acc = 0.8305 | test acc ema = 0.8330
# Epoch 290 | time = 136s | loss gen = 0.2173 | loss lab = 0.1021 | loss unl = 0.0048 | train acc = 0.9997| test acc = 0.8222 | test acc ema = 0.8340
# Epoch 291 | time = 136s | loss gen = 0.2191 | loss lab = 0.1021 | loss unl = 0.0046 | train acc = 0.9997| test acc = 0.8295 | test acc ema = 0.8339
# Epoch 292 | time = 136s | loss gen = 0.2181 | loss lab = 0.1020 | loss unl = 0.0048 | train acc = 0.9996| test acc = 0.8254 | test acc ema = 0.8343
# Epoch 293 | time = 136s | loss gen = 0.2175 | loss lab = 0.1016 | loss unl = 0.0048 | train acc = 0.9996| test acc = 0.8066 | test acc ema = 0.8356
# Epoch 294 | time = 136s | loss gen = 0.2165 | loss lab = 0.1017 | loss unl = 0.0046 | train acc = 0.9997| test acc = 0.8210 | test acc ema = 0.8350
# Epoch 295 | time = 136s | loss gen = 0.2157 | loss lab = 0.1004 | loss unl = 0.0047 | train acc = 0.9997| test acc = 0.8332 | test acc ema = 0.8341
# Epoch 296 | time = 136s | loss gen = 0.2164 | loss lab = 0.0998 | loss unl = 0.0042 | train acc = 0.9997| test acc = 0.8177 | test acc ema = 0.8340
# Epoch 297 | time = 136s | loss gen = 0.2163 | loss lab = 0.1023 | loss unl = 0.0049 | train acc = 0.9995| test acc = 0.8189 | test acc ema = 0.8336
# Epoch 298 | time = 136s | loss gen = 0.2157 | loss lab = 0.0998 | loss unl = 0.0043 | train acc = 0.9997| test acc = 0.8240 | test acc ema = 0.8337
# Epoch 299 | time = 136s | loss gen = 0.2170 | loss lab = 0.0965 | loss unl = 0.0043 | train acc = 0.9996| test acc = 0.8139 | test acc ema = 0.8339
# Epoch 300 | time = 136s | loss gen = 0.2158 | loss lab = 0.0998 | loss unl = 0.0047 | train acc = 0.9995| test acc = 0.8164 | test acc ema = 0.8333
# Model saved in file: ./log/cifar/model-300
# Epoch 301 | time = 136s | loss gen = 0.2167 | loss lab = 0.0977 | loss unl = 0.0043 | train acc = 0.9997| test acc = 0.8229 | test acc ema = 0.8349
# Epoch 302 | time = 136s | loss gen = 0.2154 | loss lab = 0.1008 | loss unl = 0.0044 | train acc = 0.9997| test acc = 0.8134 | test acc ema = 0.8346
# Epoch 303 | time = 136s | loss gen = 0.2174 | loss lab = 0.0959 | loss unl = 0.0043 | train acc = 0.9997| test acc = 0.8269 | test acc ema = 0.8355
# Epoch 304 | time = 136s | loss gen = 0.2159 | loss lab = 0.0988 | loss unl = 0.0042 | train acc = 0.9998| test acc = 0.8293 | test acc ema = 0.8347
# Epoch 305 | time = 136s | loss gen = 0.2150 | loss lab = 0.1005 | loss unl = 0.0043 | train acc = 0.9998| test acc = 0.8209 | test acc ema = 0.8354
# Epoch 306 | time = 136s | loss gen = 0.2155 | loss lab = 0.0988 | loss unl = 0.0044 | train acc = 0.9996| test acc = 0.8289 | test acc ema = 0.8355
# Epoch 307 | time = 136s | loss gen = 0.2152 | loss lab = 0.1005 | loss unl = 0.0046 | train acc = 0.9996| test acc = 0.8199 | test acc ema = 0.8361
# Epoch 308 | time = 136s | loss gen = 0.2167 | loss lab = 0.1001 | loss unl = 0.0045 | train acc = 0.9998| test acc = 0.8132 | test acc ema = 0.8356
# Epoch 309 | time = 136s | loss gen = 0.2147 | loss lab = 0.1005 | loss unl = 0.0045 | train acc = 0.9996| test acc = 0.8230 | test acc ema = 0.8361
# Epoch 310 | time = 136s | loss gen = 0.2142 | loss lab = 0.0998 | loss unl = 0.0043 | train acc = 0.9998| test acc = 0.8276 | test acc ema = 0.8368
# Epoch 311 | time = 136s | loss gen = 0.2161 | loss lab = 0.0985 | loss unl = 0.0043 | train acc = 0.9997| test acc = 0.8203 | test acc ema = 0.8368
# Epoch 312 | time = 136s | loss gen = 0.2167 | loss lab = 0.1007 | loss unl = 0.0045 | train acc = 0.9996| test acc = 0.8171 | test acc ema = 0.8366
# Epoch 313 | time = 136s | loss gen = 0.2150 | loss lab = 0.0998 | loss unl = 0.0044 | train acc = 0.9996| test acc = 0.8189 | test acc ema = 0.8366
# Epoch 314 | time = 136s | loss gen = 0.2162 | loss lab = 0.0988 | loss unl = 0.0041 | train acc = 0.9997| test acc = 0.8264 | test acc ema = 0.8356
# Epoch 315 | time = 136s | loss gen = 0.2152 | loss lab = 0.0980 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8247 | test acc ema = 0.8345
# Epoch 316 | time = 136s | loss gen = 0.2148 | loss lab = 0.0986 | loss unl = 0.0042 | train acc = 0.9997| test acc = 0.8287 | test acc ema = 0.8354
# Epoch 317 | time = 136s | loss gen = 0.2155 | loss lab = 0.0975 | loss unl = 0.0043 | train acc = 0.9998| test acc = 0.8285 | test acc ema = 0.8349
# Epoch 318 | time = 136s | loss gen = 0.2146 | loss lab = 0.0990 | loss unl = 0.0043 | train acc = 0.9996| test acc = 0.8234 | test acc ema = 0.8354
# Epoch 319 | time = 136s | loss gen = 0.2151 | loss lab = 0.0984 | loss unl = 0.0044 | train acc = 0.9996| test acc = 0.8249 | test acc ema = 0.8353
# Epoch 320 | time = 136s | loss gen = 0.2153 | loss lab = 0.0983 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8234 | test acc ema = 0.8353
# Epoch 321 | time = 136s | loss gen = 0.2141 | loss lab = 0.0995 | loss unl = 0.0043 | train acc = 0.9997| test acc = 0.8301 | test acc ema = 0.8351
# Epoch 322 | time = 136s | loss gen = 0.2152 | loss lab = 0.0982 | loss unl = 0.0039 | train acc = 0.9999| test acc = 0.8298 | test acc ema = 0.8359
# Epoch 323 | time = 136s | loss gen = 0.2139 | loss lab = 0.0998 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8221 | test acc ema = 0.8363
# Epoch 324 | time = 136s | loss gen = 0.2165 | loss lab = 0.0969 | loss unl = 0.0043 | train acc = 0.9997| test acc = 0.8248 | test acc ema = 0.8361
# Epoch 325 | time = 136s | loss gen = 0.2165 | loss lab = 0.0951 | loss unl = 0.0044 | train acc = 0.9996| test acc = 0.8312 | test acc ema = 0.8351
# Epoch 326 | time = 136s | loss gen = 0.2150 | loss lab = 0.0949 | loss unl = 0.0043 | train acc = 0.9995| test acc = 0.8301 | test acc ema = 0.8358
# Epoch 327 | time = 136s | loss gen = 0.2150 | loss lab = 0.0973 | loss unl = 0.0042 | train acc = 0.9997| test acc = 0.8254 | test acc ema = 0.8348
# Epoch 328 | time = 136s | loss gen = 0.2137 | loss lab = 0.0974 | loss unl = 0.0042 | train acc = 0.9997| test acc = 0.8303 | test acc ema = 0.8352
# Epoch 329 | time = 136s | loss gen = 0.2151 | loss lab = 0.0953 | loss unl = 0.0045 | train acc = 0.9995| test acc = 0.8241 | test acc ema = 0.8351
# Epoch 330 | time = 137s | loss gen = 0.2128 | loss lab = 0.0986 | loss unl = 0.0041 | train acc = 0.9997| test acc = 0.8307 | test acc ema = 0.8346
# Epoch 331 | time = 137s | loss gen = 0.2148 | loss lab = 0.0982 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8218 | test acc ema = 0.8347
# Epoch 332 | time = 137s | loss gen = 0.2147 | loss lab = 0.0952 | loss unl = 0.0044 | train acc = 0.9994| test acc = 0.8253 | test acc ema = 0.8348
# Epoch 333 | time = 136s | loss gen = 0.2137 | loss lab = 0.0984 | loss unl = 0.0044 | train acc = 0.9996| test acc = 0.8237 | test acc ema = 0.8351
# Epoch 334 | time = 137s | loss gen = 0.2145 | loss lab = 0.0968 | loss unl = 0.0042 | train acc = 0.9997| test acc = 0.8263 | test acc ema = 0.8358
# Epoch 335 | time = 137s | loss gen = 0.2153 | loss lab = 0.0956 | loss unl = 0.0041 | train acc = 0.9997| test acc = 0.8288 | test acc ema = 0.8365
# Epoch 336 | time = 137s | loss gen = 0.2143 | loss lab = 0.0960 | loss unl = 0.0038 | train acc = 0.9999| test acc = 0.8296 | test acc ema = 0.8360
# Epoch 337 | time = 137s | loss gen = 0.2129 | loss lab = 0.0972 | loss unl = 0.0041 | train acc = 0.9997| test acc = 0.8260 | test acc ema = 0.8368
# Epoch 338 | time = 137s | loss gen = 0.2155 | loss lab = 0.0963 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8185 | test acc ema = 0.8363
# Epoch 339 | time = 137s | loss gen = 0.2125 | loss lab = 0.0985 | loss unl = 0.0042 | train acc = 0.9995| test acc = 0.8308 | test acc ema = 0.8365
# Epoch 340 | time = 137s | loss gen = 0.2130 | loss lab = 0.0969 | loss unl = 0.0043 | train acc = 0.9996| test acc = 0.8285 | test acc ema = 0.8365
# Epoch 341 | time = 137s | loss gen = 0.2111 | loss lab = 0.0961 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8218 | test acc ema = 0.8349
# Epoch 342 | time = 137s | loss gen = 0.2135 | loss lab = 0.0952 | loss unl = 0.0043 | train acc = 0.9996| test acc = 0.8170 | test acc ema = 0.8354
# Epoch 343 | time = 137s | loss gen = 0.2110 | loss lab = 0.0954 | loss unl = 0.0040 | train acc = 0.9998| test acc = 0.8258 | test acc ema = 0.8360
# Epoch 344 | time = 137s | loss gen = 0.2136 | loss lab = 0.0941 | loss unl = 0.0041 | train acc = 0.9996| test acc = 0.8227 | test acc ema = 0.8351
# Epoch 345 | time = 137s | loss gen = 0.2130 | loss lab = 0.0946 | loss unl = 0.0042 | train acc = 0.9997| test acc = 0.8291 | test acc ema = 0.8361
# Epoch 346 | time = 137s | loss gen = 0.2132 | loss lab = 0.0927 | loss unl = 0.0039 | train acc = 0.9997| test acc = 0.8263 | test acc ema = 0.8350
# Epoch 347 | time = 137s | loss gen = 0.2134 | loss lab = 0.0941 | loss unl = 0.0040 | train acc = 0.9997| test acc = 0.8309 | test acc ema = 0.8363
# Epoch 348 | time = 137s | loss gen = 0.2144 | loss lab = 0.0932 | loss unl = 0.0040 | train acc = 0.9998| test acc = 0.8278 | test acc ema = 0.8355
# Epoch 349 | time = 137s | loss gen = 0.2126 | loss lab = 0.0948 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8287 | test acc ema = 0.8354
# Epoch 350 | time = 137s | loss gen = 0.2117 | loss lab = 0.0951 | loss unl = 0.0040 | train acc = 0.9997| test acc = 0.8265 | test acc ema = 0.8360
# Model saved in file: ./log/cifar/model-350
# Epoch 351 | time = 137s | loss gen = 0.2125 | loss lab = 0.0971 | loss unl = 0.0042 | train acc = 0.9996| test acc = 0.8292 | test acc ema = 0.8354
# Epoch 352 | time = 137s | loss gen = 0.2124 | loss lab = 0.0934 | loss unl = 0.0039 | train acc = 0.9996| test acc = 0.8335 | test acc ema = 0.8360
# Epoch 353 | time = 137s | loss gen = 0.2132 | loss lab = 0.0914 | loss unl = 0.0041 | train acc = 0.9995| test acc = 0.8270 | test acc ema = 0.8366
# Epoch 354 | time = 137s | loss gen = 0.2109 | loss lab = 0.0938 | loss unl = 0.0038 | train acc = 0.9998| test acc = 0.8315 | test acc ema = 0.8372
# Epoch 355 | time = 137s | loss gen = 0.2106 | loss lab = 0.0942 | loss unl = 0.0040 | train acc = 0.9998| test acc = 0.8228 | test acc ema = 0.8374
# Epoch 356 | time = 137s | loss gen = 0.2133 | loss lab = 0.0936 | loss unl = 0.0037 | train acc = 0.9998| test acc = 0.8229 | test acc ema = 0.8374
# Epoch 357 | time = 137s | loss gen = 0.2132 | loss lab = 0.0914 | loss unl = 0.0040 | train acc = 0.9996| test acc = 0.8276 | test acc ema = 0.8358
# Epoch 358 | time = 137s | loss gen = 0.2119 | loss lab = 0.0943 | loss unl = 0.0039 | train acc = 0.9998| test acc = 0.8351 | test acc ema = 0.8363
# Epoch 359 | time = 137s | loss gen = 0.2117 | loss lab = 0.0938 | loss unl = 0.0039 | train acc = 0.9997| test acc = 0.8274 | test acc ema = 0.8366
# Epoch 360 | time = 137s | loss gen = 0.2117 | loss lab = 0.0930 | loss unl = 0.0039 | train acc = 0.9996| test acc = 0.8270 | test acc ema = 0.8373
# Epoch 361 | time = 137s | loss gen = 0.2129 | loss lab = 0.0937 | loss unl = 0.0039 | train acc = 0.9996| test acc = 0.8255 | test acc ema = 0.8374
# Epoch 362 | time = 136s | loss gen = 0.2119 | loss lab = 0.0946 | loss unl = 0.0041 | train acc = 0.9998| test acc = 0.8243 | test acc ema = 0.8373
# Epoch 363 | time = 137s | loss gen = 0.2126 | loss lab = 0.0924 | loss unl = 0.0038 | train acc = 0.9997| test acc = 0.8290 | test acc ema = 0.8371